# **METHODE CIH-E-AD**

***Ce notebook implémente la méthode de stéganographie sans couverture basée sur la décomposition en valeurs propres, telle que décrite par Fatimah Shamsulddin Abdulsattar.***

**Étape 0 : Configuration de l'Environnement**

Nous commençons par installer les bibliothèques nécessaires. NumPy et OpenCV sont essentiels pour les manipulations d'images et les calculs matriciels. PyCryptodome est utilisé pour le chiffrement du fichier de localisation.

In [ ]:
# Bloc 1: Installation des dépendances
!pip install numpy opencv-python pycryptodome matplotlib
print("Bibliothèques installées avec succès.")


## **Étape 1 : Importations et Téléchargement de l'Image de Test**

Nous importons les modules requis et téléchargeons une image de test. L'image "Lenna" est un bon candidat car elle contient à la fois des zones lisses et des zones texturées.

In [ ]:
# Bloc 2: Importations et téléchargement de l'image
import cv2
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import os
import matplotlib.pyplot as plt
import pickle # Pour sauvegarder/charger le fichier de localisation

# Télécharger une image de test
!wget https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png -O lenna.png

# Charger l'image en niveaux de gris et la redimensionner pour la cohérence
IMG_SIZE = 512
img = cv2.imread('lenna.png', cv2.IMREAD_GRAYSCALE )
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

print(f"Image 'lenna.png' chargée et redimensionnée en {IMG_SIZE}x{IMG_SIZE}.")
plt.imshow(img, cmap='gray')
plt.title("Image de Test Originale A-ED")
plt.axis('off')
plt.show()


## **Étape 2 : Définition des Fonctions de Base**

Nous définissons ici les fonctions utilitaires pour la division en blocs, le chiffrement, et le calcul de la valeur propre maximale.

In [ ]:
# Bloc 3: Fonctions utilitaires

def get_overlapping_blocks(image, block_size, overlap_ratio):
    """
    Extrait des blocs superposés d'une image.
    L'overlap_ratio détermine le pas (stride).
    """
    stride = int(block_size * (1 - overlap_ratio))
    if stride == 0:
        stride = 1 # Assurer une progression

    h, w = image.shape
    blocks = []
    locations = []
    for y in range(0, h - block_size + 1, stride):
        for x in range(0, w - block_size + 1, stride):
            block = image[y:y+block_size, x:x+block_size]
            blocks.append(block)
            locations.append((y, x)) # Sauvegarder la coordonnée (y,x) du coin supérieur gauche
    return blocks, locations

def get_largest_eigenvalue(block):
    """
    Calcule la plus grande valeur propre d'un bloc d'image.
    """
    if block.size == 0:
        return 0
    # La matrice de covariance est nécessaire pour les valeurs propres
    # Pour simplifier, on peut calculer sur le bloc lui-même,
    # mais la covariance est plus robuste.
    # np.cov attend des données 1D, donc on aplatit le bloc.
    cov_matrix = np.cov(block.flatten())
    # Pour une seule ligne de données, cov est un scalaire. On gère ce cas.
    if np.isscalar(cov_matrix):
        return cov_matrix

    eigenvalues, _ = np.linalg.eig(np.atleast_2d(cov_matrix))
    return np.max(eigenvalues)

def encrypt_aes_gcm(data, key):
    """Chiffre des données avec AES en mode GCM."""
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(data)
    return cipher.nonce, tag, ciphertext

def decrypt_aes_gcm(nonce, tag, ciphertext, key):
    """Déchiffre des données avec AES en mode GCM."""
    try:
        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
        decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)
        return decrypted_data
    except (ValueError, KeyError):
        return None

print("Fonctions utilitaires définies.")


## **Étape 3 : Implémentation du Calcul de Hachage A-ED**


C'est le cœur de la méthode. Nous implémentons la logique de subdivision en 9 sous-blocs et la comparaison des valeurs propres pour générer le code de hachage de 8 bits.

In [ ]:
# Bloc 4: Calcul du code de hachage A-ED

def calculate_aed_hash(main_block):
    """
    Calcule le code de hachage de 8 bits pour un bloc principal
    en utilisant la méthode A-ED.
    """
    # 1. Diviser le bloc principal en 9 sous-blocs
    sub_block_size = main_block.shape[0] // 3
    sub_blocks = []
    for i in range(3):
        for j in range(3):
            sub_block = main_block[i*sub_block_size:(i+1)*sub_block_size, j*sub_block_size:(j+1)*sub_block_size]
            sub_blocks.append(sub_block)

    # 2. Calculer la plus grande valeur propre pour chaque sous-bloc
    eigenvalues = [get_largest_eigenvalue(sb) for sb in sub_blocks]

    # 3. Générer le code de hachage de 8 bits par comparaison
    # L'article original propose 4 arrangements. Nous implémentons l'arrangement 2,
    # qui est souvent un bon compromis.
    # Le principe est de comparer des paires de voisins.
    # H[0] = eig[0] > eig[1]
    # H[1] = eig[1] > eig[2]
    # H[2] = eig[3] > eig[4]
    # ... etc.

    hash_bits = []
    # Comparaisons horizontales
    hash_bits.append('1' if eigenvalues[0] > eigenvalues[1] else '0')
    hash_bits.append('1' if eigenvalues[1] > eigenvalues[2] else '0')
    hash_bits.append('1' if eigenvalues[3] > eigenvalues[4] else '0')
    hash_bits.append('1' if eigenvalues[4] > eigenvalues[5] else '0')
    hash_bits.append('1' if eigenvalues[6] > eigenvalues[7] else '0')
    hash_bits.append('1' if eigenvalues[7] > eigenvalues[8] else '0')
    # Comparaisons verticales
    hash_bits.append('1' if eigenvalues[0] > eigenvalues[3] else '0')
    hash_bits.append('1' if eigenvalues[3] > eigenvalues[6] else '0')

    hash_code = "".join(hash_bits)
    return hash_code

print("Fonction de hachage A-ED définie.")


## **Étape 4 : Phase de Dissimulation (Encodeur A-ED)**

Nous assemblons maintenant toutes les fonctions pour créer le processus complet de dissimulation.

In [ ]:
# Bloc 5: Encodeur A-ED

def hide_message_aed(image, secret_message, block_size, overlap_ratio, aes_key):
    """Dissimule un message dans une image en utilisant la méthode A-ED."""
    print("1. Extraction des blocs superposés...")
    blocks, locations = get_overlapping_blocks(image, block_size, overlap_ratio)
    print(f"   -> {len(blocks)} blocs extraits.")

    print("2. Génération de la table de correspondance (cela peut prendre du temps)...")
    lookup_table = {} # Clé: caractère (ex: 'A'), Valeur: liste d'indices de blocs
    for i, block in enumerate(blocks):
        # Calculer le hachage et le convertir en caractère ASCII
        hash_code_binary = calculate_aed_hash(block)
        char_code = int(hash_code_binary, 2)

        # S'assurer que le code est un caractère imprimable pour la démonstration
        if 32 <= char_code <= 126:
            character = chr(char_code)
            if character not in lookup_table:
                lookup_table[character] = []
            lookup_table[character].append(i)

    print("   -> Table de correspondance générée.")
    print(f"   -> {len(lookup_table)} caractères uniques trouvés dans l'image.")

    print("3. Mise en correspondance du message...")
    location_indices = []
    used_indices = set()

    for char_to_hide in secret_message:
        if char_to_hide not in lookup_table:
            print(f"ERREUR: Le caractère '{char_to_hide}' ne peut pas être représenté par cette image.")
            return None, None

        found = False
        for block_index in lookup_table[char_to_hide]:
            if block_index not in used_indices:
                location_indices.append(block_index)
                used_indices.add(block_index)
                found = True
                break

        if not found:
            print(f"ERREUR: Capacité insuffisante. Pas assez de blocs uniques pour le caractère '{char_to_hide}'.")
            return None, None

    print("   -> Message mis en correspondance avec succès.")

    print("4. Création et chiffrement du fichier de localisation...")
    # Le fichier de localisation contient les coordonnées (y,x) des blocs utilisés
    final_locations = [locations[i] for i in location_indices]

    # Sérialiser et chiffrer les données
    location_data_bytes = pickle.dumps(final_locations)
    encrypted_loc_file = encrypt_aes_gcm(location_data_bytes, aes_key)
    print("   -> Fichier de localisation chiffré.")

    return image, encrypted_loc_file

print("Encodeur A-ED défini.")


## **Étape 5 : Phase d'Extraction (Décodeur A-ED)**

Nous implémentons le processus inverse pour extraire le message.

In [ ]:
# Bloc 6: Décodeur A-ED

def extract_message_aed(image, encrypted_loc_file, block_size, aes_key):
    """Extrait un message d'une image en utilisant la méthode A-ED."""
    print("1. Déchiffrement du fichier de localisation...")
    nonce, tag, ciphertext = encrypted_loc_file
    decrypted_loc_bytes = decrypt_aes_gcm(nonce, tag, ciphertext, aes_key)

    if decrypted_loc_bytes is None:
        return "ERREUR: Échec du déchiffrement."

    # Désérialiser pour obtenir la liste des coordonnées
    locations = pickle.loads(decrypted_loc_bytes)
    print(f"   -> {len(locations)} localisations de blocs récupérées.")

    print("2. Régénération des hachages et reconstruction du message...")
    extracted_message = []
    for y, x in locations:
        # Extraire le bloc correspondant de l'image reçue
        block = image[y:y+block_size, x:x+block_size]

        # Recalculer son hachage
        hash_code_binary = calculate_aed_hash(block)
        char_code = int(hash_code_binary, 2)

        if 32 <= char_code <= 126:
            extracted_message.append(chr(char_code))
        else:
            extracted_message.append('?') # Caractère de remplacement si non-imprimable

    print("   -> Message reconstruit.")
    return "".join(extracted_message)

print("Décodeur A-ED défini.")


## **Étape 6 : Exécution du Cycle Complet**

Mettons maintenant la méthode à l'épreuve en exécutant un cycle complet de dissimulation et d'extraction.

In [ ]:
# Bloc 7: Exécution du cycle complet A-ED

# --- Paramètres ---
BLOCK_SIZE = 24          # Doit être un multiple de 3
OVERLAP_RATIO = 2/3      # Chevauchement de 2/3 comme dans l'article
SECRET_MESSAGE = "Test AED" # Message court, car la capacité peut être limitée
AES_KEY = get_random_bytes(16) # Clé AES 128-bit

# --- PHASE DE DISSIMULATION ---
print("--- DÉBUT DE LA PHASE DE DISSIMULATION A-ED ---")
stego_image_aed, encrypted_data_aed = hide_message_aed(
    img, SECRET_MESSAGE, BLOCK_SIZE, OVERLAP_RATIO, AES_KEY
)

# --- PHASE D'EXTRACTION ---
if stego_image_aed is not None:
    print("\n--- DÉBUT DE LA PHASE D'EXTRACTION A-ED ---")

    # Test 1: Extraction depuis l'image non altérée
    extracted_message_aed = extract_message_aed(
        stego_image_aed, encrypted_data_aed, BLOCK_SIZE, AES_KEY
    )

    print("\n--- RÉSULTATS ---")
    print(f"Message Original : '{SECRET_MESSAGE}'")
    print(f"Message Extrait  : '{extracted_message_aed}'")

    if SECRET_MESSAGE == extracted_message_aed:
        print("\nSUCCÈS : Le message a été extrait correctement de l'image non altérée !")
    else:
        print("\nÉCHEC : Le message extrait ne correspond pas.")

    # Test 2: Extraction depuis une image attaquée (Bruit Gaussien)
    print("\n--- TEST DE ROBUSTESSE (BRUIT GAUSSIEN) ---")
    # Créer une copie de l'image pour l'attaque
    attacked_image = stego_image_aed.copy()
    # Ajouter un bruit gaussien
    noise = np.random.normal(0, 5, attacked_image.shape).astype(np.uint8)
    attacked_image = cv2.add(attacked_image, noise)

    plt.imshow(attacked_image, cmap='gray')
    plt.title("Image après Attaque par Bruit Gaussien (sigma=5)")
    plt.axis('off')
    plt.show()

    # Tenter d'extraire le message de l'image attaquée
    extracted_message_attacked = extract_message_aed(
        attacked_image, encrypted_data_aed, BLOCK_SIZE, AES_KEY
    )
    print(f"\nMessage Extrait après Attaque : '{extracted_message_attacked}'")

    # Calcul du BER
    original_bits = ''.join(f'{ord(c):08b}' for c in SECRET_MESSAGE)
    extracted_bits = ''.join(f'{ord(c):08b}' for c in extracted_message_attacked)
    errors = sum(1 for i in range(len(original_bits)) if original_bits[i] != extracted_bits[i])
    ber = (errors / len(original_bits)) * 100
    print(f"Taux d'Erreur Binaire (BER) après attaque : {ber:.2f}%")



## **🛡️ TEST COMPLET DE LA METHODE E-AD**

## **Étape 0 : Configuration et Implémentation de la Méthode A-ED**

Nous allons d'abord réutiliser le code d'implémentation de la méthode A-ED que nous avons développé précédemment.

In [ ]:
# Bloc 1: Installation, importations et téléchargement de l'image
!pip install pycryptodome opencv-python numpy matplotlib scikit-image
import cv2
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import os
import matplotlib.pyplot as plt
import pickle
import time
from skimage.transform import rotate, rescale
from skimage.util import random_noise
from skimage.exposure import adjust_gamma, equalize_hist
import json
from datetime import datetime

# Télécharger et préparer l'image
!wget https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png -O lenna.png
IMG_SIZE = 512
BASE_IMAGE = cv2.imread('lenna.png', cv2.IMREAD_GRAYSCALE )
BASE_IMAGE = cv2.resize(BASE_IMAGE, (IMG_SIZE, IMG_SIZE))

print("Environnement prêt.")

# Bloc 2: Code complet de la méthode A-ED (regroupé pour la clarté)

# --- Fonctions Utilitaires ---
def get_overlapping_blocks(image, block_size, overlap_ratio):
    stride = int(block_size * (1 - overlap_ratio))
    if stride == 0: stride = 1
    h, w = image.shape
    blocks, locations = [], []
    for y in range(0, h - block_size + 1, stride):
        for x in range(0, w - block_size + 1, stride):
            blocks.append(image[y:y+block_size, x:x+block_size])
            locations.append((y, x))
    return blocks, locations

def get_largest_eigenvalue(block):
    if block.size < 2: return 0
    cov_matrix = np.cov(block.flatten())
    if np.isscalar(cov_matrix): return cov_matrix
    eigenvalues, _ = np.linalg.eig(np.atleast_2d(cov_matrix))
    return np.max(eigenvalues.real) # Utiliser .real pour éviter les nombres complexes dus aux imprécisions

def encrypt_aes_gcm(data, key):
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(data)
    return cipher.nonce, tag, ciphertext

def decrypt_aes_gcm(nonce, tag, ciphertext, key):
    try:
        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
        return cipher.decrypt_and_verify(ciphertext, tag)
    except (ValueError, KeyError):
        return None

# --- Cœur de la méthode A-ED ---
def calculate_aed_hash(main_block):
    sub_block_size = main_block.shape[0] // 3
    sub_blocks = []
    for i in range(3):
        for j in range(3):
            sub_blocks.append(main_block[i*sub_block_size:(i+1)*sub_block_size, j*sub_block_size:(j+1)*sub_block_size])
    eigenvalues = [get_largest_eigenvalue(sb) for sb in sub_blocks]
    hash_bits = [
        '1' if eigenvalues[0] > eigenvalues[1] else '0', '1' if eigenvalues[1] > eigenvalues[2] else '0',
        '1' if eigenvalues[3] > eigenvalues[4] else '0', '1' if eigenvalues[4] > eigenvalues[5] else '0',
        '1' if eigenvalues[6] > eigenvalues[7] else '0', '1' if eigenvalues[7] > eigenvalues[8] else '0',
        '1' if eigenvalues[0] > eigenvalues[3] else '0', '1' if eigenvalues[3] > eigenvalues[6] else '0'
    ]
    return "".join(hash_bits)

# --- Fonctions Encodeur/Décodeur ---
def hide_message_aed(image, secret_message, block_size, overlap_ratio, aes_key):
    blocks, locations = get_overlapping_blocks(image, block_size, overlap_ratio)
    lookup_table = {}
    for i, block in enumerate(blocks):
        hash_code_binary = calculate_aed_hash(block)
        char_code = int(hash_code_binary, 2)
        if 32 <= char_code <= 126:
            character = chr(char_code)
            if character not in lookup_table: lookup_table[character] = []
            lookup_table[character].append(i)
    location_indices, used_indices = [], set()
    for char_to_hide in secret_message:
        if char_to_hide not in lookup_table: return None, None, "Char not found"
        found = False
        for block_index in lookup_table[char_to_hide]:
            if block_index not in used_indices:
                location_indices.append(block_index)
                used_indices.add(block_index)
                found = True
                break
        if not found: return None, None, "Capacity issue"
    final_locations = [locations[i] for i in location_indices]
    location_data_bytes = pickle.dumps(final_locations)
    encrypted_loc_file = encrypt_aes_gcm(location_data_bytes, aes_key)
    return image, encrypted_loc_file, "Success"

def extract_message_aed(image, encrypted_loc_file, block_size, aes_key):
    nonce, tag, ciphertext = encrypted_loc_file
    decrypted_loc_bytes = decrypt_aes_gcm(nonce, tag, ciphertext, aes_key)
    if decrypted_loc_bytes is None: return None
    locations = pickle.loads(decrypted_loc_bytes)
    extracted_message = []
    for y, x in locations:
        if y + block_size > image.shape[0] or x + block_size > image.shape[1]:
            extracted_message.append('?') # Le bloc est hors des limites (après rotation/scaling)
            continue
        block = image[y:y+block_size, x:x+block_size]
        hash_code_binary = calculate_aed_hash(block)
        char_code = int(hash_code_binary, 2)
        if 32 <= char_code <= 126:
            extracted_message.append(chr(char_code))
        else:
            extracted_message.append('?')
    return "".join(extracted_message)

print("Implémentation de la méthode A-ED prête.")


**Étape 1 : Définition de la Suite de Tests**

Nous allons maintenant créer les fonctions qui exécuteront chaque test et rapporteront les résultats.

In [ ]:
# Bloc 3: Fonctions de la suite de tests

# --- Paramètres globaux pour les tests ---
BLOCK_SIZE = 24
OVERLAP_RATIO = 2/3
AES_KEY = get_random_bytes(16)
results = {}

def calculate_ber(original_msg, extracted_msg):
    """Calcule le Bit Error Rate (BER)."""
    if not extracted_msg: return 100.0
    original_bits = ''.join(f'{ord(c):08b}' for c in original_msg)
    extracted_bits = ''.join(f'{ord(c):08b}' for c in extracted_msg)

    errors = 0
    min_len = min(len(original_bits), len(extracted_bits))
    errors += sum(1 for i in range(min_len) if original_bits[i] != extracted_bits[i])
    errors += abs(len(original_bits) - len(extracted_bits)) # Gère les messages tronqués

    return (errors / len(original_bits)) * 100 if len(original_bits) > 0 else 0


# Dans le Bloc 3 (Fonctions de la suite de tests)

def run_single_test(message, attacked_image):
    """Exécute un cycle de dissimulation/extraction sur une image (potentiellement attaquée)."""
    # La dissimulation se fait toujours sur l'image de base

    # L'appel de fonction est corrigé ici (aed au lieu de aéd)
    _, encrypted_data, status = hide_message_aed(BASE_IMAGE, message, BLOCK_SIZE, OVERLAP_RATIO, AES_KEY)

    if status != "Success":
        # Gérer le cas où la dissimulation échoue (capacité insuffisante, etc.)
        # On peut considérer cela comme une erreur de 100%
        return 100.0

    # L'extraction se fait sur l'image attaquée
    extracted_msg = extract_message_aed(attacked_image, encrypted_data, BLOCK_SIZE, AES_KEY)

    return calculate_ber(message, extracted_msg)




# --- Définitions des tests ---

def test_functional():
    print("\n1. TEST FONCTIONNEL DE BASE")
    messages = ["A", "Test AED", "Message de test un peu plus long pour la methode", "Message beaucoup plus long pour verifier la capacite de base et la robustesse fonctionnelle de l'implementation complete."]
    success_count = 0
    results['functional'] = []
    for i, msg in enumerate(messages):
        ber = run_single_test(msg, BASE_IMAGE.copy())
        status = "SUCCÈS" if ber == 0.0 else "ÉCHEC"
        if status == "SUCCÈS": success_count += 1
        print(f"  Message {i+1}: {len(msg)} chars -> {status}")
        results['functional'].append({'message_length': len(msg), 'status': status})
    print(f"  Taux de réussite: {success_count / len(messages) * 100:.1f}%")

def test_jpeg_robustness():
    print("\n2. TEST ROBUSTESSE JPEG")
    message = "Test JPEG"
    results['jpeg'] = []
    for quality in [95, 80, 70, 50, 30]:
        _, buffer = cv2.imencode('.jpg', BASE_IMAGE, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
        attacked_image = cv2.imdecode(buffer, cv2.IMREAD_GRAYSCALE)
        ber = run_single_test(message, attacked_image)
        print(f"  Qualité {quality}: BER={ber:.2f}%")
        results['jpeg'].append({'quality': quality, 'ber': ber})

def test_noise_robustness():
    print("\n3. TESTS DE BRUIT")
    message = "Test Bruit"
    results['noise'] = []
    # Bruit Gaussien
    for sigma in [0.005, 0.01, 0.02, 0.05]:
        attacked_image = (random_noise(BASE_IMAGE, mode='gaussian', var=sigma) * 255).astype(np.uint8)
        ber = run_single_test(message, attacked_image)
        print(f"  Bruit Gaussien σ²={sigma}: BER={ber:.2f}%")
        results['noise'].append({'type': 'gaussian', 'sigma_sq': sigma, 'ber': ber})
    # Bruit Sel & Poivre
    for amount in [0.01, 0.03, 0.05, 0.1]:
        attacked_image = (random_noise(BASE_IMAGE, mode='s&p', amount=amount) * 255).astype(np.uint8)
        ber = run_single_test(message, attacked_image)
        print(f"  Sel & Poivre {amount*100:.0f}%: BER={ber:.2f}%")
        results['noise'].append({'type': 's&p', 'amount': amount, 'ber': ber})

def test_filtering_robustness():
    print("\n4. TESTS DE FILTRAGE")
    message = "Test Filtre"
    results['filtering'] = []
    # Flou Gaussien
    for k in [3, 5, 7, 9]:
        attacked_image = cv2.GaussianBlur(BASE_IMAGE, (k, k), 0)
        ber = run_single_test(message, attacked_image)
        print(f"  Flou Gaussien {k}x{k}: BER={ber:.2f}%")
        results['filtering'].append({'type': 'gaussian_blur', 'kernel': k, 'ber': ber})
    # Flou Médian
    for k in [3, 5, 7]:
        attacked_image = cv2.medianBlur(BASE_IMAGE, k)
        ber = run_single_test(message, attacked_image)
        print(f"  Flou Médian {k}x{k}: BER={ber:.2f}%")
        results['filtering'].append({'type': 'median_blur', 'kernel': k, 'ber': ber})

def test_geometric_robustness():
    print("\n5. TESTS GÉOMÉTRIQUES")
    message = "Test Geo"
    results['geometric'] = []
    # Rotation
    for angle in [0.5, 1, 2, 5]:
        attacked_image = (rotate(BASE_IMAGE, angle, resize=False, preserve_range=True)).astype(np.uint8)
        ber = run_single_test(message, attacked_image)
        print(f"  Rotation {angle}°: BER={ber:.2f}%")
        results['geometric'].append({'type': 'rotation', 'angle': angle, 'ber': ber})
    # Scaling
    for scale in [0.9, 1.1, 0.8, 1.2]:
        h, w = BASE_IMAGE.shape
        attacked_image = rescale(BASE_IMAGE, scale, preserve_range=True).astype(np.uint8)
        # Recadrer ou padd pour revenir à la taille originale
        nh, nw = attacked_image.shape
        if scale < 1.0: # Padding
            pad_h, pad_w = (h - nh) // 2, (w - nw) // 2
            attacked_image = np.pad(attacked_image, ((pad_h, h-nh-pad_h), (pad_w, w-nw-pad_w)), 'constant')
        else: # Recadrage
            crop_h, crop_w = (nh - h) // 2, (nw - w) // 2
            attacked_image = attacked_image[crop_h:crop_h+h, crop_w:crop_w+w]
        ber = run_single_test(message, attacked_image)
        print(f"  Scaling {scale}: BER={ber:.2f}%")
        results['geometric'].append({'type': 'scaling', 'scale': scale, 'ber': ber})

def test_contrast_luminance():
    print("\n6. TESTS CONTRASTE/LUMINOSITÉ")
    message = "Test Contraste"
    results['contrast'] = []
    # Correction Gamma
    for gamma in [0.7, 0.8, 1.2, 1.5]:
        attacked_image = (adjust_gamma(BASE_IMAGE, gamma) * 255).astype(np.uint8)
        ber = run_single_test(message, attacked_image)
        print(f"  Gamma {gamma}: BER={ber:.2f}%")
        results['contrast'].append({'type': 'gamma', 'value': gamma, 'ber': ber})
    # Luminosité
    for val in [-30, -15, 15, 30]:
        attacked_image = cv2.add(BASE_IMAGE, val)
        ber = run_single_test(message, attacked_image)
        print(f"  Luminosité {val}: BER={ber:.2f}%")
        results['contrast'].append({'type': 'brightness', 'value': val, 'ber': ber})
    # Égalisation d'histogramme
    attacked_image = equalize_hist(BASE_IMAGE)
    ber = run_single_test(message, attacked_image)
    print(f"  Égalisation histogramme: BER={ber:.2f}%")
    results['contrast'].append({'type': 'hist_equalization', 'ber': ber})

def test_capacity():
    print("\n7. TEST DE CAPACITÉ")
    results['capacity'] = {}
    blocks, _ = get_overlapping_blocks(BASE_IMAGE, BLOCK_SIZE, OVERLAP_RATIO)
    total_blocks = len(blocks)
    print(f"  Blocs totaux ({BLOCK_SIZE}x{BLOCK_SIZE}, overlap={OVERLAP_RATIO:.2f}): {total_blocks}")
    results['capacity']['total_blocks'] = total_blocks

    # La capacité théorique est le nombre de blocs, car chaque bloc représente 1 char (8 bits)
    capacity_bits = total_blocks * 8
    capacity_bytes = total_blocks
    print(f"  Capacité théorique: {capacity_bits} bits ({capacity_bytes} bytes)")
    results['capacity']['theoretical_bits'] = capacity_bits

    print("  Distribution des hachages (caractères représentables):")
    lookup_table = {}
    for i, block in enumerate(blocks):
        hash_code_binary = calculate_aed_hash(block)
        char_code = int(hash_code_binary, 2)
        if 32 <= char_code <= 126:
            character = chr(char_code)
            if character not in lookup_table: lookup_table[character] = 0
            lookup_table[character] += 1
    results['capacity']['hash_distribution'] = lookup_table
    for char, count in sorted(lookup_table.items()):
        print(f"    '{char}' (code {ord(char)}): {count} blocs")

def test_performance():
    print("\n8. TESTS DE PERFORMANCE")
    message = "Test de performance avec un message de taille raisonnable pour la methode A-ED."
    results['performance'] = {}

    # Temps de dissimulation
    start_time = time.time()
    stego_img, encrypted_data, status = hide_message_aed(BASE_IMAGE, message, BLOCK_SIZE, OVERLAP_RATIO, AES_KEY)
    end_time = time.time()
    dissimulation_time = (end_time - start_time) * 1000 # en ms
    print(f"  Temps dissimulation: {dissimulation_time:.2f} ms")
    results['performance']['dissimulation_ms'] = dissimulation_time

    # Temps d'extraction
    if status == "Success":
        start_time = time.time()
        _ = extract_message_aed(stego_img, encrypted_data, BLOCK_SIZE, AES_KEY)
        end_time = time.time()
        extraction_time = (end_time - start_time) * 1000 # en ms
        print(f"  Temps extraction: {extraction_time:.2f} ms")
        results['performance']['extraction_ms'] = extraction_time
        debit = len(message) / (dissimulation_time / 1000)
        print(f"  Débit (dissimulation): {debit:.2f} chars/sec")
        results['performance']['throughput_chars_sec'] = debit
    else:
        print(f"  Échec de la dissimulation pour le test de performance: {status}")
        results['performance']['extraction_ms'] = None
        results['performance']['throughput_chars_sec'] = None


def test_security():
    print("\n9. TESTS DE SÉCURITÉ")
    message = "Test Securite"
    results['security'] = []

    # Bonne clé
    stego_img, encrypted_data, status = hide_message_aed(BASE_IMAGE, message, BLOCK_SIZE, OVERLAP_RATIO, AES_KEY)
    if status == "Success":
        extracted_msg = extract_message_aed(stego_img, encrypted_data, BLOCK_SIZE, AES_KEY)
        status_good_key = "SUCCÈS" if message == extracted_msg else "ÉCHEC"
        print(f"  Bonne clé: {status_good_key}")
        results['security'].append({'key': 'correct', 'status': status_good_key})

        # Mauvaise clé
        wrong_key = get_random_bytes(16)
        extracted_msg_wrong = extract_message_aed(stego_img, encrypted_data, BLOCK_SIZE, wrong_key)
        status_wrong_key = "ÉCHEC" if extracted_msg_wrong is None else "SUCCÈS (inattendu)"
        print(f"  Mauvaise clé: {status_wrong_key}")
        results['security'].append({'key': 'wrong', 'status': status_wrong_key})
    else:
        print(f"  Échec de la dissimulation pour le test de sécurité: {status}")
        results['security'].append({'key': 'correct', 'status': "ÉCHEC (dissimulation failed)"})
        results['security'].append({'key': 'wrong', 'status': "Skipped (dissimulation failed)"})


print("Suite de tests pour A-ED définie.")

**Étape 2 : Exécution de la Suite de Tests Complète**

Cette dernière cellule exécute tous les tests définis ci-dessus et affiche un résumé final, tout en sauvegardant les résultats dans un fichier JSON pour une analyse ultérieure.

In [ ]:
# Bloc 4: Exécution de tous les tests et sauvegarde des résultats

print("============================================================")
print("       LANCEMENT DE LA SUITE DE TESTS POUR A-ED")
print("============================================================")

test_functional()
test_jpeg_robustness()
test_noise_robustness()
test_filtering_robustness()
test_geometric_robustness()
test_contrast_luminance()
test_capacity()
test_performance()
test_security()

# --- Sauvegarde des résultats ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"aed_test_results_{timestamp}.json"
with open(filename, 'w') as f:
    json.dump(results, f, indent=2)

print("\n============================================================")
print("                   RÉSUMÉ DES RÉSULTATS")
print("============================================================")
functional_success_rate = sum(1 for r in results['functional'] if r['status'] == 'SUCCÈS') / len(results['functional']) * 100
print(f"Taux de réussite fonctionnel: {functional_success_rate:.1f}%")
print(f"Résultats sauvegardés dans: {filename}")


# **METHODE CIHLHF**

***Ce notebook implémente la méthode de stéganographie sans couverture basée sur les valeurs de pixels les plus basses et les plus hautes d'un fragment, telle que décrite par Anggriani et al.***

**Étape 0 : Configuration de l'Environnement**

Nous commençons par installer les bibliothèques nécessaires. NumPy et OpenCV sont essentiels pour les manipulations d'images. PyCryptodome n'est pas strictement nécessaire pour la méthode de base (qui utilise une simple permutation comme clé), mais nous l'inclurons pour montrer comment sécuriser la clé de mappage si nécessaire.

In [ ]:
# Bloc 1: Installation des dépendances
!pip install numpy opencv-python pycryptodome matplotlib
print("Bibliothèques installées avec succès.")


**Étape 1 : Importations et Téléchargement de l'Image de Test**

In [ ]:
# Bloc 2: Importations et téléchargement de l'image
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import time

# Télécharger une image de test
!wget https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png -O lenna.png

# Charger l'image en niveaux de gris et la redimensionner pour la cohérence
IMG_SIZE = 512
img = cv2.imread('lenna.png', cv2.IMREAD_GRAYSCALE )
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

print(f"Image 'lenna.png' chargée et redimensionnée en {IMG_SIZE}x{IMG_SIZE}.")
plt.imshow(img, cmap='gray')
plt.title("Image de Test Originale CIHLHF")
plt.axis('off')
plt.show()


**Étape 2 : Définition des Fonctions de Base**

Nous définissons ici les fonctions utilitaires pour la division en fragments et la gestion de la clé de mappage. La clé de mappage est une permutation des indices des fragments.

In [ ]:
# Bloc 3: Fonctions utilitaires

def split_image_into_fragments(image, fragment_size):
    """Divise une image en une liste de fragments non superposés."""
    h, w = image.shape
    fragments = []
    for y in range(0, h, fragment_size):
        for x in range(0, w, fragment_size):
            if y + fragment_size <= h and x + fragment_size <= w:
                fragments.append(image[y:y+fragment_size, x:x+fragment_size])
    return fragments

def generate_mapping_key(num_fragments, seed=None):
    """
    Génère une clé de mappage, qui est une permutation des indices des fragments.
    Un 'seed' (graine) est utilisé pour rendre la permutation reproductible.
    Le seed est la clé secrète partagée.
    """
    indices = list(range(num_fragments))
    if seed is not None:
        random.seed(seed)
    random.shuffle(indices)
    return indices

print("Fonctions utilitaires définies.")


**Étape 3 : Implémentation du Cœur de la Méthode CIHLHF**

C'est ici que nous codons la logique principale de la méthode : l'extraction des valeurs min/max et la génération du code de hachage de 2 bits.

In [ ]:
# Bloc 4: Calcul du code de hachage CIHLHF

def get_min_max_values(fragment):
    """Retourne les valeurs minimale et maximale d'un fragment."""
    if fragment.size == 0:
        return 0, 0
    return np.min(fragment), np.max(fragment)

def get_msb(value):
    """Retourne le bit de poids fort (MSB) d'une valeur de 8 bits."""
    return '1' if value >= 128 else '0'

def calculate_cihlhf_hash(fragment):
    """
    Calcule le code de hachage de 2 bits pour un fragment
    en utilisant la méthode CIHLHF.
    """
    min_val, max_val = get_min_max_values(fragment)
    msb_min = get_msb(min_val)
    msb_max = get_msb(max_val)

    # Le hachage est la concaténation du MSB du min et du MSB du max
    return f"{msb_min}{msb_max}"

print("Fonction de hachage CIHLHF définie.")


**Étape 4 : Phase de Dissimulation (Encodeur CIHLHF)**

Nous assemblons les fonctions pour créer le processus complet de dissimulation.

In [ ]:
# Bloc 5: Encodeur CIHLHF

def hide_message_cihlhf(image, secret_message, fragment_size, mapping_key_seed):
    """Dissimule un message dans une image en utilisant la méthode CIHLHF."""
    # 1. Préparation
    fragments = split_image_into_fragments(image, fragment_size)
    num_fragments = len(fragments)

    # Le message est converti en binaire
    message_binary = ''.join(f'{ord(c):08b}' for c in secret_message)

    # La capacité est de 2 bits par fragment
    capacity_bits = num_fragments * 2
    if len(message_binary) > capacity_bits:
        print(f"ERREUR: Le message est trop long. Capacité max: {capacity_bits} bits, Message: {len(message_binary)} bits.")
        return None, None

    # 2. Génération de la clé de mappage
    mapping_key = generate_mapping_key(num_fragments, seed=mapping_key_seed)

    # 3. Génération de la table de correspondance (Lookup Table)
    # C'est une simple liste de hachages dans l'ordre des fragments
    all_hashes = [calculate_cihlhf_hash(f) for f in fragments]

    # 4. Création du "Mapping Flag" (indicateur de mappage)
    # Le mapping flag indique si le bit du message correspond au bit de hachage.
    # U = NOT(T XOR H)
    # U=1 si T==H, U=0 si T!=H

    mapping_flag = []
    message_bits_used = 0

    # La dissimulation se fait en utilisant la clé de mappage pour parcourir les fragments
    for fragment_index in mapping_key:
        if message_bits_used >= len(message_binary):
            break # Tout le message a été caché

        hash_bits = all_hashes[fragment_index] # Hachage de 2 bits du fragment

        # Cacher le premier bit du message
        message_bit_1 = message_binary[message_bits_used]
        flag_1 = '1' if message_bit_1 == hash_bits[0] else '0'
        mapping_flag.append(flag_1)
        message_bits_used += 1

        if message_bits_used >= len(message_binary):
            break

        # Cacher le second bit du message
        message_bit_2 = message_binary[message_bits_used]
        flag_2 = '1' if message_bit_2 == hash_bits[1] else '0'
        mapping_flag.append(flag_2)
        message_bits_used += 1

    print(f"Message dissimulé avec succès. Longueur du mapping flag: {len(mapping_flag)} bits.")
    return image, "".join(mapping_flag)

print("Encodeur CIHLHF défini.")


**Étape 5 : Phase d'Extraction (Décodeur CIHLHF)**

Nous implémentons le processus inverse pour extraire le message.

In [ ]:
# Bloc 6: Décodeur CIHLHF

def extract_message_cihlhf(image, mapping_flag, fragment_size, mapping_key_seed):
    """Extrait un message d'une image en utilisant la méthode CIHLHF."""
    # 1. Préparation
    fragments = split_image_into_fragments(image, fragment_size)
    num_fragments = len(fragments)

    # 2. Régénération de la clé de mappage et des hachages
    mapping_key = generate_mapping_key(num_fragments, seed=mapping_key_seed)
    all_hashes = [calculate_cihlhf_hash(f) for f in fragments]

    # 3. Reconstruction du message
    extracted_binary = []
    flag_index = 0

    for fragment_index in mapping_key:
        if flag_index >= len(mapping_flag):
            break # On a lu tout le mapping flag

        hash_bits = all_hashes[fragment_index]

        # Extraire le premier bit
        flag_1 = mapping_flag[flag_index]
        # Si flag=1, T=H. Si flag=0, T=NOT(H)
        extracted_bit_1 = hash_bits[0] if flag_1 == '1' else ('0' if hash_bits[0] == '1' else '1')
        extracted_binary.append(extracted_bit_1)
        flag_index += 1

        if flag_index >= len(mapping_flag):
            break

        # Extraire le second bit
        flag_2 = mapping_flag[flag_index]
        extracted_bit_2 = hash_bits[1] if flag_2 == '1' else ('0' if hash_bits[1] == '1' else '1')
        extracted_binary.append(extracted_bit_2)
        flag_index += 1

    # 4. Conversion du binaire en texte
    binary_string = "".join(extracted_binary)
    secret_message = ""
    for i in range(0, len(binary_string), 8):
        byte = binary_string[i:i+8]
        if len(byte) == 8:
            try:
                secret_message += chr(int(byte, 2))
            except ValueError:
                secret_message += '?' # Caractère invalide

    return secret_message

print("Décodeur CIHLHF défini.")


**Étape 6 : Exécution du Cycle Complet**

Mettons la méthode à l'épreuve en exécutant un cycle complet de dissimulation et d'extraction.

In [ ]:
# Bloc 7: Exécution du cycle complet CIHLHF

# --- Paramètres ---
FRAGMENT_SIZE = 8
SECRET_MESSAGE = "Test de la methode CIHLHF qui utilise les valeurs min et max."
# La clé secrète est la "graine" pour le générateur de nombres aléatoires
SECRET_KEY_SEED = 42

# --- PHASE DE DISSIMULATION ---
print("--- DÉBUT DE LA PHASE DE DISSIMULATION CIHLHF ---")
stego_image_cihlhf, mapping_flag_cihlhf = hide_message_cihlhf(
    img, SECRET_MESSAGE, FRAGMENT_SIZE, SECRET_KEY_SEED
)

# --- PHASE D'EXTRACTION ---
if stego_image_cihlhf is not None:
    print("\n--- DÉBUT DE LA PHASE D'EXTRACTION CIHLHF ---")

    # Test 1: Extraction depuis l'image non altérée
    extracted_message_cihlhf = extract_message_cihlhf(
        stego_image_cihlhf, mapping_flag_cihlhf, FRAGMENT_SIZE, SECRET_KEY_SEED
    )

    print("\n--- RÉSULTATS ---")
    print(f"Message Original : '{SECRET_MESSAGE}'")
    print(f"Message Extrait  : '{extracted_message_cihlhf}'")

    if SECRET_MESSAGE == extracted_message_cihlhf:
        print("\nSUCCÈS : Le message a été extrait correctement de l'image non altérée !")
    else:
        print("\nÉCHEC : Le message extrait ne correspond pas.")

    # Test 2: Extraction depuis une image attaquée (Bruit Sel & Poivre)
    # C'est l'attaque la plus pertinente pour cette méthode.
    print("\n--- TEST DE ROBUSTESSE (BRUIT SEL & POIVRE) ---")

    # Créer une copie de l'image pour l'attaque
    attacked_image = img.copy()
    # Ajouter un bruit "sel et poivre" de seulement 1%
    noise_amount = 0.01
    num_salt = np.ceil(noise_amount * attacked_image.size * 0.5)
    num_pepper = np.ceil(noise_amount * attacked_image.size * 0.5)

    # Poivre (0)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in attacked_image.shape]
    attacked_image[coords[0], coords[1]] = 0
    # Sel (255)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in attacked_image.shape]
    attacked_image[coords[0], coords[1]] = 255

    plt.imshow(attacked_image, cmap='gray')
    plt.title(f"Image après Attaque Sel & Poivre ({noise_amount*100}%)")
    plt.axis('off')
    plt.show()

    # Tenter d'extraire le message de l'image attaquée
    extracted_message_attacked = extract_message_cihlhf(
        attacked_image, mapping_flag_cihlhf, FRAGMENT_SIZE, SECRET_KEY_SEED
    )
    print(f"\nMessage Extrait après Attaque : '{extracted_message_attacked}'")

    # Calcul du BER
    original_bits = ''.join(f'{ord(c):08b}' for c in SECRET_MESSAGE)
    extracted_bits = ''.join(f'{ord(c):08b}' for c in extracted_message_attacked)
    errors = sum(1 for i in range(min(len(original_bits), len(extracted_bits))) if original_bits[i] != extracted_bits[i])
    errors += abs(len(original_bits) - len(extracted_bits))
    ber = (errors / len(original_bits)) * 100 if original_bits else 0
    print(f"Taux d'Erreur Binaire (BER) après attaque : {ber:.2f}%")



## **TESTS COMPLET DE LA METHODE CIHLHF**

Ce notebook a pour objectif de réaliser une évaluation exhaustive de la méthode de stéganographie CIHLHF en la soumettant à une série de tests fonctionnels, de robustesse, de capacité et de performance.

**Étape 0 : Configuration et Implémentation de la Méthode CIHLHF**

Nous allons d'abord réutiliser le code d'implémentation de la méthode CIHLHF que nous avons développé précédemment.

In [ ]:
# Bloc 1: Installation, importations et téléchargement de l'image
!pip install scikit-image opencv-python numpy matplotlib
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import time
from skimage.transform import rotate, rescale
from skimage.util import random_noise
from skimage.exposure import adjust_gamma, equalize_hist
import json
from datetime import datetime

# Télécharger et préparer l'image
!wget https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png -O lenna.png
IMG_SIZE = 512
BASE_IMAGE = cv2.imread('lenna.png', cv2.IMREAD_GRAYSCALE )
BASE_IMAGE = cv2.resize(BASE_IMAGE, (IMG_SIZE, IMG_SIZE))

print("Environnement prêt.")

# Bloc 2: Code complet de la méthode CIHLHF (regroupé pour la clarté)

# --- Fonctions Utilitaires ---
def split_image_into_fragments(image, fragment_size):
    h, w = image.shape
    fragments = []
    for y in range(0, h, fragment_size):
        for x in range(0, w, fragment_size):
            if y + fragment_size <= h and x + fragment_size <= w:
                fragments.append(image[y:y+fragment_size, x:x+fragment_size])
    return fragments

def generate_mapping_key(num_fragments, seed=None):
    indices = list(range(num_fragments))
    if seed is not None:
        random.seed(seed)
    random.shuffle(indices)
    return indices

# --- Cœur de la méthode CIHLHF ---
def get_min_max_values(fragment):
    if fragment.size == 0: return 0, 0
    return np.min(fragment), np.max(fragment)

def get_msb(value):
    return '1' if value >= 128 else '0'

def calculate_cihlhf_hash(fragment):
    min_val, max_val = get_min_max_values(fragment)
    return f"{get_msb(min_val)}{get_msb(max_val)}"

# --- Fonctions Encodeur/Décodeur ---
def hide_message_cihlhf(image, secret_message, fragment_size, mapping_key_seed):
    fragments = split_image_into_fragments(image, fragment_size)
    num_fragments = len(fragments)
    message_binary = ''.join(f'{ord(c):08b}' for c in secret_message)
    capacity_bits = num_fragments * 2
    if len(message_binary) > capacity_bits:
        return None, None, "Capacity issue"
    mapping_key = generate_mapping_key(num_fragments, seed=mapping_key_seed)
    all_hashes = [calculate_cihlhf_hash(f) for f in fragments]
    mapping_flag = []
    message_bits_used = 0
    for fragment_index in mapping_key:
        if message_bits_used >= len(message_binary): break
        hash_bits = all_hashes[fragment_index]
        for bit_in_hash in hash_bits:
            if message_bits_used >= len(message_binary): break
            message_bit = message_binary[message_bits_used]
            mapping_flag.append('1' if message_bit == bit_in_hash else '0')
            message_bits_used += 1
    return image, "".join(mapping_flag), "Success"

def extract_message_cihlhf(image, mapping_flag, fragment_size, mapping_key_seed):
    if not mapping_flag: return ""
    fragments = split_image_into_fragments(image, fragment_size)
    num_fragments = len(fragments)
    mapping_key = generate_mapping_key(num_fragments, seed=mapping_key_seed)
    all_hashes = [calculate_cihlhf_hash(f) for f in fragments]
    extracted_binary = []
    flag_index = 0
    for fragment_index in mapping_key:
        if flag_index >= len(mapping_flag): break
        hash_bits = all_hashes[fragment_index]
        for bit_in_hash in hash_bits:
            if flag_index >= len(mapping_flag): break
            flag = mapping_flag[flag_index]
            extracted_binary.append(bit_in_hash if flag == '1' else ('0' if bit_in_hash == '1' else '1'))
            flag_index += 1
    binary_string = "".join(extracted_binary)
    secret_message = ""
    for i in range(0, len(binary_string), 8):
        byte = binary_string[i:i+8]
        if len(byte) == 8:
            try: secret_message += chr(int(byte, 2))
            except ValueError: secret_message += '?'
    return secret_message

print("Implémentation de la méthode CIHLHF prête.")


**Étape 1 : Définition de la Suite de Tests**

Nous réutilisons la même structure de test que pour les méthodes précédentes, en l'adaptant pour CIHLHF.

In [ ]:
# Bloc 3: Fonctions de la suite de tests pour CIHLHF

# --- Paramètres globaux pour les tests ---
FRAGMENT_SIZE = 8
SECRET_KEY_SEED = 42 # La clé secrète partagée
results = {}

def calculate_ber(original_msg, extracted_msg):
    """Calcule le Bit Error Rate (BER)."""
    if not extracted_msg: return 100.0
    original_bits = ''.join(f'{ord(c):08b}' for c in original_msg)
    extracted_bits = ''.join(f'{ord(c):08b}' for c in extracted_msg)
    errors = 0
    min_len = min(len(original_bits), len(extracted_bits))
    errors += sum(1 for i in range(min_len) if original_bits[i] != extracted_bits[i])
    errors += abs(len(original_bits) - len(extracted_bits))
    return (errors / len(original_bits)) * 100 if len(original_bits) > 0 else 0

def run_single_test_cihlhf(message, attacked_image):
    """Exécute un cycle de dissimulation/extraction sur une image pour CIHLHF."""
    _, mapping_flag, status = hide_message_cihlhf(BASE_IMAGE, message, FRAGMENT_SIZE, SECRET_KEY_SEED)
    if status != "Success":
        return 100.0
    extracted_msg = extract_message_cihlhf(attacked_image, mapping_flag, FRAGMENT_SIZE, SECRET_KEY_SEED)
    return calculate_ber(message, extracted_msg)

# --- Définitions des tests ---

def test_functional():
    print("\n1. TEST FONCTIONNEL DE BASE")
    messages = ["A", "Test CIHLHF", "Message de test un peu plus long pour la methode", "Message beaucoup plus long pour verifier la capacite de base et la robustesse fonctionnelle de l'implementation complete."]
    success_count = 0
    results['functional'] = []
    for i, msg in enumerate(messages):
        ber = run_single_test_cihlhf(msg, BASE_IMAGE.copy())
        status = "SUCCÈS" if ber == 0.0 else "ÉCHEC"
        if status == "SUCCÈS": success_count += 1
        print(f"  Message {i+1}: {len(msg)} chars -> {status}")
        results['functional'].append({'message_length': len(msg), 'status': status})
    print(f"  Taux de réussite: {success_count / len(messages) * 100:.1f}%")

def test_jpeg_robustness():
    print("\n2. TEST ROBUSTESSE JPEG")
    message = "Test JPEG"
    results['jpeg'] = []
    for quality in [95, 80, 70, 50, 30]:
        _, buffer = cv2.imencode('.jpg', BASE_IMAGE, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
        attacked_image = cv2.imdecode(buffer, cv2.IMREAD_GRAYSCALE)
        ber = run_single_test_cihlhf(message, attacked_image)
        print(f"  Qualité {quality}: BER={ber:.2f}%")
        results['jpeg'].append({'quality': quality, 'ber': ber})

def test_noise_robustness():
    print("\n3. TESTS DE BRUIT")
    message = "Test Bruit"
    results['noise'] = []
    # Bruit Gaussien
    for sigma in [0.005, 0.01, 0.02, 0.05]:
        attacked_image = (random_noise(BASE_IMAGE, mode='gaussian', var=sigma) * 255).astype(np.uint8)
        ber = run_single_test_cihlhf(message, attacked_image)
        print(f"  Bruit Gaussien σ²={sigma}: BER={ber:.2f}%")
        results['noise'].append({'type': 'gaussian', 'sigma_sq': sigma, 'ber': ber})
    # Bruit Sel & Poivre
    for amount in [0.01, 0.03, 0.05, 0.1]:
        attacked_image = (random_noise(BASE_IMAGE, mode='s&p', amount=amount) * 255).astype(np.uint8)
        ber = run_single_test_cihlhf(message, attacked_image)
        print(f"  Sel & Poivre {amount*100:.0f}%: BER={ber:.2f}%")
        results['noise'].append({'type': 's&p', 'amount': amount, 'ber': ber})

def test_filtering_robustness():
    print("\n4. TESTS DE FILTRAGE")
    message = "Test Filtre"
    results['filtering'] = []
    # Flou Gaussien
    for k in [3, 5, 7, 9]:
        attacked_image = cv2.GaussianBlur(BASE_IMAGE, (k, k), 0)
        ber = run_single_test_cihlhf(message, attacked_image)
        print(f"  Flou Gaussien {k}x{k}: BER={ber:.2f}%")
        results['filtering'].append({'type': 'gaussian_blur', 'kernel': k, 'ber': ber})
    # Flou Médian
    for k in [3, 5, 7]:
        attacked_image = cv2.medianBlur(BASE_IMAGE, k)
        ber = run_single_test_cihlhf(message, attacked_image)
        print(f"  Flou Médian {k}x{k}: BER={ber:.2f}%")
        results['filtering'].append({'type': 'median_blur', 'kernel': k, 'ber': ber})

def test_geometric_robustness():
    print("\n5. TESTS GÉOMÉTRIQUES")
    message = "Test Geo"
    results['geometric'] = []
    # Rotation
    for angle in [0.5, 1, 2, 5]:
        attacked_image = (rotate(BASE_IMAGE, angle, resize=False, preserve_range=True)).astype(np.uint8)
        ber = run_single_test_cihlhf(message, attacked_image)
        print(f"  Rotation {angle}°: BER={ber:.2f}%")
        results['geometric'].append({'type': 'rotation', 'angle': angle, 'ber': ber})
    # Scaling
    for scale in [0.9, 1.1, 0.8, 1.2]:
        h, w = BASE_IMAGE.shape
        attacked_image = rescale(BASE_IMAGE, scale, preserve_range=True).astype(np.uint8)
        nh, nw = attacked_image.shape
        if scale < 1.0:
            pad_h, pad_w = (h - nh) // 2, (w - nw) // 2
            attacked_image = np.pad(attacked_image, ((pad_h, h-nh-pad_h), (pad_w, w-nw-pad_w)), 'constant')
        else:
            crop_h, crop_w = (nh - h) // 2, (nw - w) // 2
            attacked_image = attacked_image[crop_h:crop_h+h, crop_w:crop_w+w]
        ber = run_single_test_cihlhf(message, attacked_image)
        print(f"  Scaling {scale}: BER={ber:.2f}%")
        results['geometric'].append({'type': 'scaling', 'scale': scale, 'ber': ber})

def test_contrast_luminance():
    print("\n6. TESTS CONTRASTE/LUMINOSITÉ")
    message = "Test Contraste"
    results['contrast'] = []
    # Correction Gamma
    for gamma in [0.7, 0.8, 1.2, 1.5]:
        attacked_image = (adjust_gamma(BASE_IMAGE, gamma) * 255).astype(np.uint8)
        ber = run_single_test_cihlhf(message, attacked_image)
        print(f"  Gamma {gamma}: BER={ber:.2f}%")
        results['contrast'].append({'type': 'gamma', 'value': gamma, 'ber': ber})
    # Luminosité
    for val in [-30, -15, 15, 30]:
        attacked_image = cv2.add(BASE_IMAGE, val)
        # S'assurer que les valeurs restent dans [0, 255]
        attacked_image[BASE_IMAGE < abs(val) if val < 0 else BASE_IMAGE > 255-val] = 0 if val < 0 else 255
        ber = run_single_test_cihlhf(message, attacked_image)
        print(f"  Luminosité {val}: BER={ber:.2f}%")
        results['contrast'].append({'type': 'brightness', 'value': val, 'ber': ber})
    # Égalisation d'histogramme
    attacked_image = equalize_hist(BASE_IMAGE)
    ber = run_single_test_cihlhf(message, attacked_image)
    print(f"  Égalisation histogramme: BER={ber:.2f}%")
    results['contrast'].append({'type': 'hist_equalization', 'ber': ber})

def test_capacity():
    print("\n7. TEST DE CAPACITÉ")
    results['capacity'] = {}
    fragments = split_image_into_fragments(BASE_IMAGE, FRAGMENT_SIZE)
    total_fragments = len(fragments)
    print(f"  Fragments totaux ({FRAGMENT_SIZE}x{FRAGMENT_SIZE}): {total_fragments}")
    results['capacity']['total_fragments'] = total_fragments

    capacity_bits = total_fragments * 2
    capacity_bytes = capacity_bits // 8
    print(f"  Capacité théorique: {capacity_bits} bits ({capacity_bytes} bytes)")
    results['capacity']['theoretical_bits'] = capacity_bits

    print("  Distribution des hachages:")
    lookup_table = {'00': 0, '01': 0, '10': 0, '11': 0}
    for f in fragments:
        h = calculate_cihlhf_hash(f)
        lookup_table[h] += 1
    results['capacity']['hash_distribution'] = lookup_table
    for h, count in lookup_table.items():
        print(f"    {h}: {count} fragments")

def test_performance():
    print("\n8. TESTS DE PERFORMANCE")
    message = "Test de performance avec un message de taille raisonnable pour la methode CIHLHF."
    results['performance'] = {}

    start_time = time.time()
    _, _, _ = hide_message_cihlhf(BASE_IMAGE, message, FRAGMENT_SIZE, SECRET_KEY_SEED)
    end_time = time.time()
    dissimulation_time = (end_time - start_time) * 1000
    print(f"  Temps dissimulation: {dissimulation_time:.2f} ms")
    results['performance']['dissimulation_ms'] = dissimulation_time

    _, mapping_flag, _ = hide_message_cihlhf(BASE_IMAGE, message, FRAGMENT_SIZE, SECRET_KEY_SEED)
    start_time = time.time()
    _ = extract_message_cihlhf(BASE_IMAGE, mapping_flag, FRAGMENT_SIZE, SECRET_KEY_SEED)
    end_time = time.time()
    extraction_time = (end_time - start_time) * 1000
    print(f"  Temps extraction: {extraction_time:.2f} ms")
    results['performance']['extraction_ms'] = extraction_time

    debit = len(message) / (dissimulation_time / 1000)
    print(f"  Débit (dissimulation): {debit:.2f} chars/sec")
    results['performance']['throughput_chars_sec'] = debit

def test_security():
    print("\n9. TESTS DE SÉCURITÉ")
    message = "Test Securite"
    results['security'] = []

    _, mapping_flag, _ = hide_message_cihlhf(BASE_IMAGE, message, FRAGMENT_SIZE, SECRET_KEY_SEED)

    # Bonne clé
    extracted_msg = extract_message_cihlhf(BASE_IMAGE, mapping_flag, FRAGMENT_SIZE, SECRET_KEY_SEED)
    status_good_key = "SUCCÈS" if message == extracted_msg else "ÉCHEC"
    print(f"  Bonne clé (seed={SECRET_KEY_SEED}): {status_good_key}")
    results['security'].append({'key': 'correct', 'status': status_good_key})

    # Mauvaise clé
    wrong_key_seed = SECRET_KEY_SEED + 1
    extracted_msg_wrong = extract_message_cihlhf(BASE_IMAGE, mapping_flag, FRAGMENT_SIZE, wrong_key_seed)
    status_wrong_key = "ÉCHEC" if message != extracted_msg_wrong else "SUCCÈS (inattendu)"
    print(f"  Mauvaise clé (seed={wrong_key_seed}): {status_wrong_key}")
    results['security'].append({'key': 'wrong', 'status': status_wrong_key})

print("Suite de tests pour CIHLHF définie.")


**Étape 2 : Exécution de la Suite de Tests Complète**

Cette dernière cellule exécute tous les tests définis ci-dessus pour la méthode CIHLHF.

In [ ]:
# Bloc 4: Exécution de tous les tests et sauvegarde des résultats

print("============================================================")
print("      LANCEMENT DE LA SUITE DE TESTS POUR CIHLHF")
print("============================================================")

test_functional()
test_jpeg_robustness()
test_noise_robustness()
test_filtering_robustness()
test_geometric_robustness()
test_contrast_luminance()
test_capacity()
test_performance()
test_security()

# --- Sauvegarde des résultats ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"cihlhf_test_results_{timestamp}.json"
with open(filename, 'w') as f:
    json.dump(results, f, indent=2)

print("\n============================================================")
print("                   RÉSUMÉ DES RÉSULTATS")
print("============================================================")
functional_success_rate = sum(1 for r in results['functional'] if r['status'] == 'SUCCÈS') / len(results['functional']) * 100
print(f"Taux de réussite fonctionnel: {functional_success_rate:.1f}%")
print(f"Résultats sauvegardés dans: {filename}")


# **METHODE CIH-QR**

***Ce notebook implémente la méthode de stéganographie sans couverture CIH-QR, basée sur le classement par quantiles et les relations de voisinage.***

**Étape 0 : Configuration de l'Environnement**

In [ ]:
# Bloc 1: Installation des dépendances
!pip install numpy opencv-python pycryptodome matplotlib
print("Bibliothèques installées avec succès.")


Étape 1 : Importations et Téléchargement de l'Image de Test

Nous importons les modules requis et téléchargeons une image de test. L'image "Lenna" est un bon candidat car elle contient à la fois des zones lisses et des zones texturées.

In [ ]:
# Bloc 2: Importations et téléchargement de l'image
import cv2
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import os
import matplotlib.pyplot as plt
import pickle # Pour sauvegarder/charger le fichier de localisation
import time

# Télécharger une image de test
!wget https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png -O lenna.png

# Charger l'image en niveaux de gris et la redimensionner pour la cohérence
IMG_SIZE = 512
img = cv2.imread('lenna.png', cv2.IMREAD_GRAYSCALE )
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

print(f"Image 'lenna.png' chargée et redimensionnée en {IMG_SIZE}x{IMG_SIZE}.")
plt.imshow(img, cmap='gray')
plt.title("Image de Test Originale")
plt.axis('off')
plt.show()


**Étape 2 : Définition des Fonctions Utilitaires**

Nous définissons ici les fonctions utilitaires pour la division en fragments et le chiffrement/déchiffrement AES.

In [ ]:
# Bloc 3: Fonctions utilitaires

def split_image_into_fragments(image, fragment_size):
    """Divise une image en une liste de fragments non superposés."""
    h, w = image.shape
    fragments = []
    for y in range(0, h, fragment_size):
        for x in range(0, w, fragment_size):
            if y + fragment_size <= h and x + fragment_size <= w:
                fragments.append(image[y:y+fragment_size, x:x+fragment_size])
    return fragments

def encrypt_aes_gcm(data, key):
    """Chiffre des données avec AES en mode GCM."""
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(data)
    return cipher.nonce, tag, ciphertext

def decrypt_aes_gcm(nonce, tag, ciphertext, key):
    """Déchiffre des données avec AES en mode GCM."""
    try:
        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
        decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)
        return decrypted_data
    except (ValueError, KeyError):
        return None

print("Fonctions utilitaires définies.")


**Étape 3 : Implémentation du Cœur de la Méthode CIH-QR**

C'est ici que nous codons la logique principale de notre méthode : le calcul du code de hachage de 4 bits pour chaque fragment.

In [ ]:
# Bloc 4: Calcul du code de hachage CIH-QR

def calculate_cih_qr_hash(fragment, full_image_iqr_avg, right_neighbor_median, bottom_neighbor_median):
    """Calcule le code de hachage de 4 bits pour un fragment d'image."""
    if fragment.size == 0:
        return "0000"

    # Calcul des caractéristiques statistiques robustes
    q1, q2_median, q3 = np.percentile(fragment, [25, 50, 75])

    # Bit 1: Asymétrie interne
    b1 = '1' if (q3 - q2_median) > (q2_median - q1) else '0'

    # Bit 2: Texture locale (basée sur l'écart interquartile)
    iqr = q3 - q1
    b2 = '1' if iqr > full_image_iqr_avg else '0'

    # Bit 3: Relation de voisinage horizontal
    b3 = '1' if q2_median > right_neighbor_median else '0'

    # Bit 4: Relation de voisinage vertical
    b4 = '1' if q2_median > bottom_neighbor_median else '0'

    return f"{b1}{b2}{b3}{b4}"

def generate_all_hashes_and_lookup_table(fragments, image_width, fragment_size):
    """Génère les hachages pour tous les fragments et la table de correspondance."""
    num_fragments = len(fragments)
    cols = image_width // fragment_size

    # Pré-calculer les médianes et IQRs de tous les fragments pour l'efficacité
    all_stats = [np.percentile(f, [25, 50, 75]) for f in fragments]
    all_medians = [s[1] for s in all_stats]
    all_iqrs = [s[2] - s[0] for s in all_stats]
    full_image_iqr_avg = np.mean(all_iqrs)

    hashes = []
    lookup_table = {f'{i:04b}': [] for i in range(16)}

    for i in range(num_fragments):
        # Gérer les voisins de manière circulaire (torique) pour les bords
        current_row = i // cols
        # Voisin de droite
        right_neighbor_idx = current_row * cols + (i + 1) % cols
        # Voisin du bas
        bottom_neighbor_idx = (i + cols) % num_fragments

        right_neighbor_median = all_medians[right_neighbor_idx]
        bottom_neighbor_median = all_medians[bottom_neighbor_idx]

        # Le fragment courant est le i-ème de la liste
        current_fragment = fragments[i]

        h = calculate_cih_qr_hash(current_fragment, full_image_iqr_avg, right_neighbor_median, bottom_neighbor_median)
        hashes.append(h)
        lookup_table[h].append(i)

    return hashes, lookup_table

print("Fonction de hachage CIH-QR définie.")


**Étape 4 : Phase de Dissimulation (Encodeur CIH-QR)**

Nous assemblons maintenant toutes les fonctions pour créer le processus complet de dissimulation.

In [ ]:
# Bloc 5: Encodeur CIH-QR

def hide_message_cih_qr(image, secret_message, fragment_size, aes_key):
    """Dissimule un message dans une image en utilisant la méthode CIH-QR."""
    # 1. Préparation
    fragments = split_image_into_fragments(image, fragment_size)
    message_binary = ''.join(f'{ord(c):08b}' for c in secret_message)

    # S'assurer que le message binaire est un multiple de 4 (taille de nos hachages)
    if len(message_binary) % 4 != 0:
        padding = '0' * (4 - len(message_binary) % 4)
        message_binary += padding

    # 2. Génération des hachages et de la table
    _, lookup_table = generate_all_hashes_and_lookup_table(fragments, image.shape[1], fragment_size)

    print("Distribution des hachages générés :")
    for h_val, indices in sorted(lookup_table.items()):
        print(f"  - Hachage '{h_val}': {len(indices)} fragments disponibles")

    # 3. Mise en correspondance
    location_indices = []
    used_indices = set()

    for i in range(0, len(message_binary), 4):
        segment = message_binary[i:i+4]

        if segment not in lookup_table or not lookup_table[segment]:
            print(f"ERREUR: Aucun fragment disponible pour le hachage '{segment}'.")
            return None, None

        found_index = -1
        for index in lookup_table[segment]:
            if index not in used_indices:
                found_index = index
                break

        if found_index == -1:
            print(f"ERREUR: Capacité insuffisante. Pas assez de blocs uniques pour le segment '{segment}'.")
            return None, None

        location_indices.append(found_index)
        used_indices.add(found_index)

    # 4. Sécurisation
    location_data_bytes = pickle.dumps(location_indices)
    encrypted_loc_file = encrypt_aes_gcm(location_data_bytes, aes_key)

    print(f"\nMessage dissimulé avec succès ! {len(location_indices)} fragments utilisés.")
    return image, encrypted_loc_file

print("Encodeur CIH-QR défini.")


**Étape 5 : Phase d'Extraction (Décodeur CIH-QR)**

Nous implémentons le processus inverse pour extraire le message.

In [ ]:
# Bloc 6: Décodeur CIH-QR

def extract_message_cih_qr(image, encrypted_loc_file, fragment_size, aes_key):
    """Extrait un message d'une image en utilisant la méthode CIH-QR."""
    # 1. Déchiffrement
    nonce, tag, ciphertext = encrypted_loc_file
    decrypted_loc_bytes = decrypt_aes_gcm(nonce, tag, ciphertext, aes_key)

    if decrypted_loc_bytes is None:
        return "ERREUR: Échec du déchiffrement."

    location_indices = pickle.loads(decrypted_loc_bytes)

    # 2. Régénération des hachages
    fragments = split_image_into_fragments(image, fragment_size)
    all_hashes, _ = generate_all_hashes_and_lookup_table(fragments, image.shape[1], fragment_size)

    # 3. Reconstruction
    extracted_binary = ""
    for index in location_indices:
        if index < len(all_hashes):
            extracted_binary += all_hashes[index]
        else:
            extracted_binary += "????" # Erreur, l'indice est hors limites

    # 4. Conversion du binaire en texte
    secret_message = ""
    for i in range(0, len(extracted_binary), 8):
        byte = extracted_binary[i:i+8]
        if len(byte) == 8:
            try:
                secret_message += chr(int(byte, 2))
            except ValueError:
                secret_message += '?'

    return secret_message.strip('\x00') # Retirer les caractères nuls du padding

print("Décodeur CIH-QR défini.")


**Étape 6 : Exécution du Cycle Complet**

Mettons notre méthode à l'épreuve en exécutant un cycle complet de dissimulation et d'extraction, incluant un test de robustesse.

In [ ]:
# Bloc 7: Exécution du cycle complet CIH-QR

# --- Paramètres ---
FRAGMENT_SIZE = 8
SECRET_MESSAGE = "Test de la methode CIH-QR, basee sur les quantiles et les relations de voisinage."
AES_KEY = get_random_bytes(16)

# --- PHASE DE DISSIMULATION ---
print("--- DÉBUT DE LA PHASE DE DISSIMULATION CIH-QR ---")
stego_image_cihqr, encrypted_data_cihqr = hide_message_cih_qr(
    img, SECRET_MESSAGE, FRAGMENT_SIZE, AES_KEY
)

# --- PHASE D'EXTRACTION ---
if stego_image_cihqr is not None:
    print("\n--- DÉBUT DE LA PHASE D'EXTRACTION CIH-QR ---")

    # Test 1: Extraction depuis l'image non altérée
    extracted_message_cihqr = extract_message_cih_qr(
        stego_image_cihqr, encrypted_data_cihqr, FRAGMENT_SIZE, AES_KEY
    )

    print("\n--- RÉSULTATS ---")
    print(f"Message Original : '{SECRET_MESSAGE}'")
    print(f"Message Extrait  : '{extracted_message_cihqr}'")

    if SECRET_MESSAGE == extracted_message_cihqr:
        print("\nSUCCÈS : Le message a été extrait correctement de l'image non altérée !")
    else:
        print("\nÉCHEC : Le message extrait ne correspond pas.")

    # Test 2: Extraction depuis une image attaquée (Bruit Sel & Poivre)
    print("\n--- TEST DE ROBUSTESSE (BRUIT SEL & POIVRE) ---")

    attacked_image = img.copy()
    noise_amount = 0.01
    # Utiliser la fonction de scikit-image pour un bruit plus réaliste
    from skimage.util import random_noise
    attacked_image = (random_noise(attacked_image, mode='s&p', amount=noise_amount) * 255).astype(np.uint8)

    plt.imshow(attacked_image, cmap='gray')
    plt.title(f"Image après Attaque Sel & Poivre ({noise_amount*100}%)")
    plt.axis('off')
    plt.show()

    # Tenter d'extraire le message de l'image attaquée
    extracted_message_attacked = extract_message_cih_qr(
        attacked_image, encrypted_data_cihqr, FRAGMENT_SIZE, AES_KEY
    )
    print(f"\nMessage Extrait après Attaque : '{extracted_message_attacked}'")

    # Calcul du BER
    original_bits = ''.join(f'{ord(c):08b}' for c in SECRET_MESSAGE)
    extracted_bits = ''.join(f'{ord(c):08b}' for c in extracted_message_attacked)
    errors = sum(1 for i in range(min(len(original_bits), len(extracted_bits))) if original_bits[i] != extracted_bits[i])
    errors += abs(len(original_bits) - len(extracted_bits))
    ber = (errors / len(original_bits)) * 100 if original_bits else 0
    print(f"Taux d'Erreur Binaire (BER) après attaque : {ber:.2f}%")


## **TEST COMPLET DE LA METHODE CIH-QR**

Ce notebook a pour objectif de réaliser une évaluation exhaustive de notre méthode de stéganographie CIH-QR en la soumettant à une série de tests fonctionnels, de robustesse, de capacité et de performance.

**Étape 0 : Configuration et Implémentation de la Méthode CIH-QR**

Nous allons d'abord réutiliser le code d'implémentation de la méthode CIH-QR que nous avons développé précédemment.

In [ ]:
# Bloc 1: Installation, importations et téléchargement de l'image
!pip install scikit-image opencv-python numpy matplotlib pycryptodome
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import time
from skimage.transform import rotate, rescale
from skimage.util import random_noise
from skimage.exposure import adjust_gamma, equalize_hist
import json
from datetime import datetime

# Télécharger et préparer l'image
!wget https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png -O lenna.png
IMG_SIZE = 512
BASE_IMAGE = cv2.imread('lenna.png', cv2.IMREAD_GRAYSCALE )
BASE_IMAGE = cv2.resize(BASE_IMAGE, (IMG_SIZE, IMG_SIZE))

print("Environnement prêt.")

# Bloc 2: Code complet de la méthode CIH-QR (regroupé pour la clarté)

# --- Fonctions Utilitaires ---
def split_image_into_fragments(image, fragment_size):
    h, w = image.shape
    fragments = []
    for y in range(0, h, fragment_size):
        for x in range(0, w, fragment_size):
            if y + fragment_size <= h and x + fragment_size <= w:
                fragments.append(image[y:y+fragment_size, x:x+fragment_size])
    return fragments

def encrypt_aes_gcm(data, key):
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(data)
    return cipher.nonce, tag, ciphertext

def decrypt_aes_gcm(nonce, tag, ciphertext, key):
    try:
        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
        return cipher.decrypt_and_verify(ciphertext, tag)
    except (ValueError, KeyError):
        return None

# --- Cœur de la méthode CIH-QR ---
def calculate_cih_qr_hash(fragment, full_image_iqr_avg, right_neighbor_median, bottom_neighbor_median):
    if fragment.size == 0: return "0000"
    q1, q2_median, q3 = np.percentile(fragment, [25, 50, 75])
    b1 = '1' if (q3 - q2_median) > (q2_median - q1) else '0'
    iqr = q3 - q1
    b2 = '1' if iqr > full_image_iqr_avg else '0'
    b3 = '1' if q2_median > right_neighbor_median else '0'
    b4 = '1' if q2_median > bottom_neighbor_median else '0'
    return f"{b1}{b2}{b3}{b4}"

def generate_all_hashes_and_lookup_table(fragments, image_width, fragment_size):
    num_fragments = len(fragments)
    cols = image_width // fragment_size
    all_stats = [np.percentile(f, [25, 50, 75]) for f in fragments]
    all_medians = [s[1] for s in all_stats]
    all_iqrs = [s[2] - s[0] for s in all_stats]
    full_image_iqr_avg = np.mean(all_iqrs) if all_iqrs else 0
    hashes, lookup_table = [], {f'{i:04b}': [] for i in range(16)}
    for i in range(num_fragments):
        current_row = i // cols
        right_neighbor_idx = current_row * cols + (i + 1) % cols
        bottom_neighbor_idx = (i + cols) % num_fragments
        h = calculate_cih_qr_hash(fragments[i], full_image_iqr_avg, all_medians[right_neighbor_idx], all_medians[bottom_neighbor_idx])
        hashes.append(h)
        lookup_table[h].append(i)
    return hashes, lookup_table

# --- Fonctions Encodeur/Décodeur ---
def hide_message_cih_qr(image, secret_message, fragment_size, aes_key):
    fragments = split_image_into_fragments(image, fragment_size)
    message_binary = ''.join(f'{ord(c):08b}' for c in secret_message)
    if len(message_binary) % 4 != 0:
        message_binary += '0' * (4 - len(message_binary) % 4)
    _, lookup_table = generate_all_hashes_and_lookup_table(fragments, image.shape[1], fragment_size)
    location_indices, used_indices = [], set()
    for i in range(0, len(message_binary), 4):
        segment = message_binary[i:i+4]
        if not lookup_table.get(segment): return None, None, f"No fragments for hash {segment}"
        found_index = -1
        for index in lookup_table[segment]:
            if index not in used_indices:
                found_index = index
                break
        if found_index == -1: return None, None, f"Capacity issue for hash {segment}"
        location_indices.append(found_index)
        used_indices.add(found_index)
    location_data_bytes = pickle.dumps(location_indices)
    encrypted_loc_file = encrypt_aes_gcm(location_data_bytes, aes_key)
    return image, encrypted_loc_file, "Success"

def extract_message_cih_qr(image, encrypted_loc_file, fragment_size, aes_key):
    if not encrypted_loc_file: return None
    nonce, tag, ciphertext = encrypted_loc_file
    decrypted_loc_bytes = decrypt_aes_gcm(nonce, tag, ciphertext, aes_key)
    if decrypted_loc_bytes is None: return None
    location_indices = pickle.loads(decrypted_loc_bytes)
    fragments = split_image_into_fragments(image, fragment_size)
    all_hashes, _ = generate_all_hashes_and_lookup_table(fragments, image.shape[1], fragment_size)
    extracted_binary = ""
    for index in location_indices:
        if index < len(all_hashes):
            extracted_binary += all_hashes[index]
        else:
            extracted_binary += "????"
    secret_message = ""
    for i in range(0, len(extracted_binary), 8):
        byte = extracted_binary[i:i+8]
        if len(byte) == 8:
            try: secret_message += chr(int(byte, 2))
            except ValueError: secret_message += '?'
    return secret_message.strip('\x00')

print("Implémentation de la méthode CIH-QR prête.")


**Étape 1 : Définition de la Suite de Tests**

Nous réutilisons la même structure de test que pour les méthodes précédentes, en l'adaptant pour CIH-QR.

In [ ]:
# Bloc 3: Fonctions de la suite de tests pour CIH-QR

# --- Paramètres globaux pour les tests ---
FRAGMENT_SIZE = 8
AES_KEY = get_random_bytes(16)
results = {}

def calculate_ber(original_msg, extracted_msg):
    """Calcule le Bit Error Rate (BER)."""
    if not extracted_msg: return 100.0
    original_bits = ''.join(f'{ord(c):08b}' for c in original_msg)
    extracted_bits = ''.join(f'{ord(c):08b}' for c in extracted_msg)
    errors = 0
    min_len = min(len(original_bits), len(extracted_bits))
    errors += sum(1 for i in range(min_len) if original_bits[i] != extracted_bits[i])
    errors += abs(len(original_bits) - len(extracted_bits))
    return (errors / len(original_bits)) * 100 if len(original_bits) > 0 else 0

def run_single_test_cihqr(message, attacked_image):
    """Exécute un cycle de dissimulation/extraction sur une image pour CIH-QR."""
    _, encrypted_data, status = hide_message_cih_qr(BASE_IMAGE, message, FRAGMENT_SIZE, AES_KEY)
    if status != "Success":
        return 100.0
    extracted_msg = extract_message_cih_qr(attacked_image, encrypted_data, FRAGMENT_SIZE, AES_KEY)
    return calculate_ber(message, extracted_msg)

# --- Définitions des tests ---

def test_functional():
    print("\n1. TEST FONCTIONNEL DE BASE")
    messages = ["A", "Test CIH-QR", "Message de test un peu plus long pour la methode", "Message beaucoup plus long pour verifier la capacite de base et la robustesse fonctionnelle de l'implementation complete."]
    success_count = 0
    results['functional'] = []
    for i, msg in enumerate(messages):
        ber = run_single_test_cihqr(msg, BASE_IMAGE.copy())
        status = "SUCCÈS" if ber == 0.0 else "ÉCHEC"
        if status == "SUCCÈS": success_count += 1
        print(f"  Message {i+1}: {len(msg)} chars -> {status}")
        results['functional'].append({'message_length': len(msg), 'status': status})
    print(f"  Taux de réussite: {success_count / len(messages) * 100:.1f}%")

def test_jpeg_robustness():
    print("\n2. TEST ROBUSTESSE JPEG")
    message = "Test JPEG"
    results['jpeg'] = []
    for quality in [95, 80, 70, 50, 30]:
        _, buffer = cv2.imencode('.jpg', BASE_IMAGE, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
        attacked_image = cv2.imdecode(buffer, cv2.IMREAD_GRAYSCALE)
        ber = run_single_test_cihqr(message, attacked_image)
        print(f"  Qualité {quality}: BER={ber:.2f}%")
        results['jpeg'].append({'quality': quality, 'ber': ber})

def test_noise_robustness():
    print("\n3. TESTS DE BRUIT")
    message = "Test Bruit"
    results['noise'] = []
    # Bruit Gaussien
    for sigma in [0.005, 0.01, 0.02, 0.05]:
        attacked_image = (random_noise(BASE_IMAGE, mode='gaussian', var=sigma) * 255).astype(np.uint8)
        ber = run_single_test_cihqr(message, attacked_image)
        print(f"  Bruit Gaussien σ²={sigma}: BER={ber:.2f}%")
        results['noise'].append({'type': 'gaussian', 'sigma_sq': sigma, 'ber': ber})
    # Bruit Sel & Poivre
    for amount in [0.01, 0.03, 0.05, 0.1]:
        attacked_image = (random_noise(BASE_IMAGE, mode='s&p', amount=amount) * 255).astype(np.uint8)
        ber = run_single_test_cihqr(message, attacked_image)
        print(f"  Sel & Poivre {amount*100:.0f}%: BER={ber:.2f}%")
        results['noise'].append({'type': 's&p', 'amount': amount, 'ber': ber})

def test_filtering_robustness():
    print("\n4. TESTS DE FILTRAGE")
    message = "Test Filtre"
    results['filtering'] = []
    # Flou Gaussien
    for k in [3, 5, 7, 9]:
        attacked_image = cv2.GaussianBlur(BASE_IMAGE, (k, k), 0)
        ber = run_single_test_cihqr(message, attacked_image)
        print(f"  Flou Gaussien {k}x{k}: BER={ber:.2f}%")
        results['filtering'].append({'type': 'gaussian_blur', 'kernel': k, 'ber': ber})
    # Flou Médian
    for k in [3, 5, 7]:
        attacked_image = cv2.medianBlur(BASE_IMAGE, k)
        ber = run_single_test_cihqr(message, attacked_image)
        print(f"  Flou Médian {k}x{k}: BER={ber:.2f}%")
        results['filtering'].append({'type': 'median_blur', 'kernel': k, 'ber': ber})

def test_geometric_robustness():
    print("\n5. TESTS GÉOMÉTRIQUES")
    message = "Test Geo"
    results['geometric'] = []
    # Rotation
    for angle in [0.5, 1, 2, 5]:
        attacked_image = (rotate(BASE_IMAGE, angle, resize=False, preserve_range=True)).astype(np.uint8)
        ber = run_single_test_cihqr(message, attacked_image)
        print(f"  Rotation {angle}°: BER={ber:.2f}%")
        results['geometric'].append({'type': 'rotation', 'angle': angle, 'ber': ber})
    # Scaling
    for scale in [0.9, 1.1, 0.8, 1.2]:
        h, w = BASE_IMAGE.shape
        attacked_image = rescale(BASE_IMAGE, scale, preserve_range=True).astype(np.uint8)
        nh, nw = attacked_image.shape
        if scale < 1.0:
            pad_h, pad_w = (h - nh) // 2, (w - nw) // 2
            attacked_image = np.pad(attacked_image, ((pad_h, h-nh-pad_h), (pad_w, w-nw-pad_w)), 'constant')
        else:
            crop_h, crop_w = (nh - h) // 2, (nw - w) // 2
            attacked_image = attacked_image[crop_h:crop_h+h, crop_w:crop_w+w]
        ber = run_single_test_cihqr(message, attacked_image)
        print(f"  Scaling {scale}: BER={ber:.2f}%")
        results['geometric'].append({'type': 'scaling', 'scale': scale, 'ber': ber})

def test_contrast_luminance():
    print("\n6. TESTS CONTRASTE/LUMINOSITÉ")
    message = "Test Contraste"
    results['contrast'] = []
    # Correction Gamma
    for gamma in [0.7, 0.8, 1.2, 1.5]:
        attacked_image = (adjust_gamma(BASE_IMAGE, gamma) * 255).astype(np.uint8)
        ber = run_single_test_cihqr(message, attacked_image)
        print(f"  Gamma {gamma}: BER={ber:.2f}%")
        results['contrast'].append({'type': 'gamma', 'value': gamma, 'ber': ber})
    # Luminosité
    for val in [-30, -15, 15, 30]:
        attacked_image = cv2.add(BASE_IMAGE, val)
        attacked_image[BASE_IMAGE < abs(val) if val < 0 else BASE_IMAGE > 255-val] = 0 if val < 0 else 255
        ber = run_single_test_cihqr(message, attacked_image)
        print(f"  Luminosité {val}: BER={ber:.2f}%")
        results['contrast'].append({'type': 'brightness', 'value': val, 'ber': ber})
    # Égalisation d'histogramme
    attacked_image = equalize_hist(BASE_IMAGE)
    ber = run_single_test_cihqr(message, attacked_image)
    print(f"  Égalisation histogramme: BER={ber:.2f}%")
    results['contrast'].append({'type': 'hist_equalization', 'ber': ber})

def test_capacity():
    print("\n7. TEST DE CAPACITÉ")
    results['capacity'] = {}
    fragments = split_image_into_fragments(BASE_IMAGE, FRAGMENT_SIZE)
    total_fragments = len(fragments)
    print(f"  Fragments totaux ({FRAGMENT_SIZE}x{FRAGMENT_SIZE}): {total_fragments}")
    results['capacity']['total_fragments'] = total_fragments

    capacity_bits = total_fragments * 4
    capacity_bytes = capacity_bits // 8
    print(f"  Capacité théorique: {capacity_bits} bits ({capacity_bytes} bytes)")
    results['capacity']['theoretical_bits'] = capacity_bits

    print("  Distribution des hachages:")
    _, lookup_table = generate_all_hashes_and_lookup_table(fragments, BASE_IMAGE.shape[1], FRAGMENT_SIZE)
    results['capacity']['hash_distribution'] = {h: len(indices) for h, indices in lookup_table.items()}
    for h, count in sorted(results['capacity']['hash_distribution'].items()):
        print(f"    {h}: {count} fragments")

def test_performance():
    print("\n8. TESTS DE PERFORMANCE")
    message = "Test de performance avec un message de taille raisonnable pour la methode CIH-QR."
    results['performance'] = {}

    start_time = time.time()
    _, _, _ = hide_message_cih_qr(BASE_IMAGE, message, FRAGMENT_SIZE, AES_KEY)
    end_time = time.time()
    dissimulation_time = (end_time - start_time) * 1000
    print(f"  Temps dissimulation: {dissimulation_time:.2f} ms")
    results['performance']['dissimulation_ms'] = dissimulation_time

    _, encrypted_data, _ = hide_message_cih_qr(BASE_IMAGE, message, FRAGMENT_SIZE, AES_KEY)
    start_time = time.time()
    _ = extract_message_cih_qr(BASE_IMAGE, encrypted_data, FRAGMENT_SIZE, AES_KEY)
    end_time = time.time()
    extraction_time = (end_time - start_time) * 1000
    print(f"  Temps extraction: {extraction_time:.2f} ms")
    results['performance']['extraction_ms'] = extraction_time

    debit = len(message) / (dissimulation_time / 1000)
    print(f"  Débit (dissimulation): {debit:.2f} chars/sec")
    results['performance']['throughput_chars_sec'] = debit

def test_security():
    print("\n9. TESTS DE SÉCURITÉ")
    message = "Test Securite"
    results['security'] = []

    _, encrypted_data, _ = hide_message_cih_qr(BASE_IMAGE, message, FRAGMENT_SIZE, AES_KEY)

    # Bonne clé
    extracted_msg = extract_message_cih_qr(BASE_IMAGE, encrypted_data, FRAGMENT_SIZE, AES_KEY)
    status_good_key = "SUCCÈS" if message == extracted_msg else "ÉCHEC"
    print(f"  Bonne clé: {status_good_key}")
    results['security'].append({'key': 'correct', 'status': status_good_key})

    # Mauvaise clé
    wrong_key = get_random_bytes(16)
    extracted_msg_wrong = extract_message_cih_qr(BASE_IMAGE, encrypted_data, FRAGMENT_SIZE, wrong_key)
    status_wrong_key = "ÉCHEC" if extracted_msg_wrong is None else "SUCCÈS (inattendu)"
    print(f"  Mauvaise clé: {status_wrong_key}")
    results['security'].append({'key': 'wrong', 'status': status_wrong_key})

print("Suite de tests pour CIH-QR définie.")


**Étape 2 : Exécution de la Suite de Tests Complète**

Cette dernière cellule exécute tous les tests définis ci-dessus pour la méthode CIH-QR.

In [ ]:
# Bloc 4: Exécution de tous les tests et sauvegarde des résultats

print("============================================================")
print("       LANCEMENT DE LA SUITE DE TESTS POUR CIH-QR")
print("============================================================")

test_functional()
test_jpeg_robustness()
test_noise_robustness()
test_filtering_robustness()
test_geometric_robustness()
test_contrast_luminance()
test_capacity()
test_performance()
test_security()

# --- Sauvegarde des résultats ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"cihqr_test_results_{timestamp}.json"
with open(filename, 'w') as f:
    json.dump(results, f, indent=2)

print("\n============================================================")
print("                   RÉSUMÉ DES RÉSULTATS")
print("============================================================")
functional_success_rate = sum(1 for r in results['functional'] if r['status'] == 'SUCCÈS') / len(results['functional']) * 100
print(f"Taux de réussite fonctionnel: {functional_success_rate:.1f}%")
print(f"Résultats sauvegardés dans: {filename}")


# **ANALYSE DES RESULTATS**

### **Analyse de Robustesse des Méthodes de Stéganographie**

**1. Robustesse à la Compression JPEG**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Données pour la compression JPEG
qualities = [95, 80, 70, 50, 30]
e_ad_jpeg = [11.11, 19.44, 22.22, 23.61, 27.78]
cihlhf_jpeg = [1.39, 1.39, 0.0, 1.39, 0.0]
cih_qr_jpeg = [2.78, 6.94, 5.56, 5.56, 5.56]

plt.figure(figsize=(10, 6))
plt.plot(qualities, e_ad_jpeg, 'ro-', linewidth=2, markersize=8, label='E-AD')
plt.plot(qualities, cihlhf_jpeg, 'bo-', linewidth=2, markersize=8, label='CIHLHF')
plt.plot(qualities, cih_qr_jpeg, 'go-', linewidth=2, markersize=8, label='CIH-QR')
plt.xlabel('Qualité JPEG')
plt.ylabel('Taux d\'erreur (BER %)')
plt.title('Robustesse à la Compression JPEG')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.gca().invert_xaxis()  # Inverser l'axe x pour que la qualité diminue de gauche à droite
plt.savefig('robustesse_jpeg.png', dpi=300, bbox_inches='tight')
plt.show()

**2. Robustesse au Bruit**

In [ ]:
# Données pour le bruit gaussien
sigma_sq = [0.005, 0.01, 0.02, 0.05]
e_ad_gaussian = [100, 100, 100, 100]
cihlhf_gaussian = [23.75, 31.25, 31.25, 35.0]
cih_qr_gaussian = [23.75, 25.0, 31.25, 26.25]

# Données pour le bruit poivre et sel
snp_amount = [0.01, 0.03, 0.05, 0.1]
e_ad_snp = [100, 100, 100, 100]
cihlhf_snp = [12.5, 20.0, 26.25, 35.0]
cih_qr_snp = [1.25, 6.25, 0.0, 5.0]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Graphique bruit gaussien
ax1.plot(sigma_sq, e_ad_gaussian, 'ro-', linewidth=2, markersize=8, label='E-AD')
ax1.plot(sigma_sq, cihlhf_gaussian, 'bo-', linewidth=2, markersize=8, label='CIHLHF')
ax1.plot(sigma_sq, cih_qr_gaussian, 'go-', linewidth=2, markersize=8, label='CIH-QR')
ax1.set_xlabel('Variance du bruit (σ²)')
ax1.set_ylabel('Taux d\'erreur (BER %)')
ax1.set_title('Robustesse au Bruit Gaussien')
ax1.legend()
ax1.grid(True, linestyle='--', alpha=0.7)

# Graphique bruit poivre et sel
ax2.plot(snp_amount, e_ad_snp, 'ro-', linewidth=2, markersize=8, label='E-AD')
ax2.plot(snp_amount, cihlhf_snp, 'bo-', linewidth=2, markersize=8, label='CIHLHF')
ax2.plot(snp_amount, cih_qr_snp, 'go-', linewidth=2, markersize=8, label='CIH-QR')
ax2.set_xlabel('Quantité de bruit')
ax2.set_ylabel('Taux d\'erreur (BER %)')
ax2.set_title('Robustesse au Bruit Poivre et Sel')
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('robustesse_bruit.png', dpi=300, bbox_inches='tight')
plt.show()

**3. Robustesse au Filtrage**

In [ ]:
# Données pour le filtrage gaussien
gaussian_kernels = [3, 5, 7, 9]
e_ad_gaussian = [100, 100, 100, 100]
cihlhf_gaussian = [2.27, 3.41, 4.55, 4.55]
cih_qr_gaussian = [7.95, 10.23, 6.82, 7.95]

# Données pour le filtrage médian
median_kernels = [3, 5, 7]
e_ad_median = [100, 100, 100]
cihlhf_median = [1.14, 3.41, 4.55]
cih_qr_median = [6.82, 4.55, 9.09]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Graphique filtrage gaussien
ax1.plot(gaussian_kernels, e_ad_gaussian, 'ro-', linewidth=2, markersize=8, label='E-AD')
ax1.plot(gaussian_kernels, cihlhf_gaussian, 'bo-', linewidth=2, markersize=8, label='CIHLHF')
ax1.plot(gaussian_kernels, cih_qr_gaussian, 'go-', linewidth=2, markersize=8, label='CIH-QR')
ax1.set_xlabel('Taille du noyau')
ax1.set_ylabel('Taux d\'erreur (BER %)')
ax1.set_title('Robustesse au Filtrage Gaussien')
ax1.legend()
ax1.grid(True, linestyle='--', alpha=0.7)

# Graphique filtrage médian
ax2.plot(median_kernels, e_ad_median, 'ro-', linewidth=2, markersize=8, label='E-AD')
ax2.plot(median_kernels, cihlhf_median, 'bo-', linewidth=2, markersize=8, label='CIHLHF')
ax2.plot(median_kernels, cih_qr_median, 'go-', linewidth=2, markersize=8, label='CIH-QR')
ax2.set_xlabel('Taille du noyau')
ax2.set_ylabel('Taux d\'erreur (BER %)')
ax2.set_title('Robustesse au Filtrage Médian')
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('robustesse_filtrage.png', dpi=300, bbox_inches='tight')
plt.show()

**4. Robustesse aux Transformations Géométriques**

In [ ]:
# Données pour la rotation
rotation_angles = [0.5, 1, 2, 5]
e_ad_rotation = [25.0, 32.81, 43.75, 54.69]
cihlhf_rotation = [3.13, 4.69, 12.5, 15.63]
cih_qr_rotation = [23.44, 46.88, 50.0, 48.44]

# Données pour le redimensionnement
scaling_factors = [0.9, 1.1, 0.8, 1.2]
e_ad_scaling = [48.44, 43.75, 48.44, 46.88]
cihlhf_scaling = [20.31, 31.25, 34.38, 37.5]
cih_qr_scaling = [95.31, 40.63, 100.0, 40.63]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Graphique rotation
ax1.plot(rotation_angles, e_ad_rotation, 'ro-', linewidth=2, markersize=8, label='E-AD')
ax1.plot(rotation_angles, cihlhf_rotation, 'bo-', linewidth=2, markersize=8, label='CIHLHF')
ax1.plot(rotation_angles, cih_qr_rotation, 'go-', linewidth=2, markersize=8, label='CIH-QR')
ax1.set_xlabel('Angle de rotation (degrés)')
ax1.set_ylabel('Taux d\'erreur (BER %)')
ax1.set_title('Robustesse à la Rotation')
ax1.legend()
ax1.grid(True, linestyle='--', alpha=0.7)

# Graphique redimensionnement
ax2.plot(scaling_factors, e_ad_scaling, 'ro-', linewidth=2, markersize=8, label='E-AD')
ax2.plot(scaling_factors, cihlhf_scaling, 'bo-', linewidth=2, markersize=8, label='CIHLHF')
ax2.plot(scaling_factors, cih_qr_scaling, 'go-', linewidth=2, markersize=8, label='CIH-QR')
ax2.set_xlabel('Facteur d\'échelle')
ax2.set_ylabel('Taux d\'erreur (BER %)')
ax2.set_title('Robustesse au Redimensionnement')
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('robustesse_geometrique.png', dpi=300, bbox_inches='tight')
plt.show()

**5. Robustesse aux Modifications de Contraste**

In [ ]:
# Données pour les corrections gamma
gamma_values = [0.7, 0.8, 1.2, 1.5]
e_ad_gamma = [100, 100, 100, 100]
cihlhf_gamma = [59.82, 63.39, 67.86, 55.36]
cih_qr_gamma = [69.64, 67.86, 66.07, 69.64]

# Données pour les modifications de luminosité
brightness_values = [-30, -15, 15, 30]
e_ad_brightness = [100, 100, 100, 100]
cihlhf_brightness = [25.89, 5.36, 6.25, 14.29]
cih_qr_brightness = [0.0, 0.0, 0.0, 0.0]

# Données pour l'égalisation d'histogramme
methods = ['E-AD', 'CIHLHF', 'CIH-QR']
hist_eq = [100, 63.39, 8.04]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Graphique corrections gamma
ax1.plot(gamma_values, e_ad_gamma, 'ro-', linewidth=2, markersize=8, label='E-AD')
ax1.plot(gamma_values, cihlhf_gamma, 'bo-', linewidth=2, markersize=8, label='CIHLHF')
ax1.plot(gamma_values, cih_qr_gamma, 'go-', linewidth=2, markersize=8, label='CIH-QR')
ax1.set_xlabel('Valeur gamma')
ax1.set_ylabel('Taux d\'erreur (BER %)')
ax1.set_title('Robustesse aux Corrections Gamma')
ax1.legend()
ax1.grid(True, linestyle='--', alpha=0.7)

# Graphique modifications de luminosité
ax2.plot(brightness_values, e_ad_brightness, 'ro-', linewidth=2, markersize=8, label='E-AD')
ax2.plot(brightness_values, cihlhf_brightness, 'bo-', linewidth=2, markersize=8, label='CIHLHF')
ax2.plot(brightness_values, cih_qr_brightness, 'go-', linewidth=2, markersize=8, label='CIH-QR')
ax2.set_xlabel('Modification de luminosité')
ax2.set_ylabel('Taux d\'erreur (BER %)')
ax2.set_title('Robustesse aux Modifications de Luminosité')
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.7)

# Graphique égalisation d'histogramme
ax3.bar(methods, hist_eq, color=['red', 'blue', 'green'], alpha=0.7)
ax3.set_ylabel('Taux d\'erreur (BER %)')
ax3.set_title('Robustesse à l\'Égalisation d\'Histogramme')
ax3.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('robustesse_contraste.png', dpi=300, bbox_inches='tight')
plt.show()

**6. Analyse Comparative Globale de Robustesse**

In [ ]:
# Calcul des scores moyens de robustesse par type d'attaque
categories = ['JPEG', 'Bruit Gaussien', 'Bruit S&P', 'Filtrage Gaussien',
              'Filtrage Médian', 'Rotation', 'Redimensionnement', 'Correction Gamma',
              'Luminosité', 'Égalisation Hist.']

# Scores moyens (0-10, 10 étant le meilleur)
e_ad_scores = [2, 0, 0, 0, 0, 5, 4, 0, 0, 0]
cihlhf_scores = [10, 7, 8, 9, 9, 9, 6, 4, 8, 4]
cih_qr_scores = [8, 7, 9, 7, 7, 5, 3, 3, 10, 9]

x = np.arange(len(categories))
width = 0.25

plt.figure(figsize=(16, 8))
plt.bar(x - width, e_ad_scores, width, label='E-AD', color='red', alpha=0.7)
plt.bar(x, cihlhf_scores, width, label='CIHLHF', color='blue', alpha=0.7)
plt.bar(x + width, cih_qr_scores, width, label='CIH-QR', color='green', alpha=0.7)

plt.xlabel('Type d\'attaque')
plt.ylabel('Score de robustesse (0-10)')
plt.title('Score Global de Robustesse par Type d\'Attaque')
plt.xticks(x, categories, rotation=45, ha='right')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7, axis='y')
plt.tight_layout()
plt.savefig('score_global_robustesse.png', dpi=300, bbox_inches='tight')
plt.show()

# Calcul du score moyen global
e_ad_mean = np.mean(e_ad_scores)
cihlhf_mean = np.mean(cihlhf_scores)
cih_qr_mean = np.mean(cih_qr_scores)

print(f"Score moyen de robustesse E-AD: {e_ad_mean:.2f}/10")
print(f"Score moyen de robustesse CIHLHF: {cihlhf_mean:.2f}/10")
print(f"Score moyen de robustesse CIH-QR: {cih_qr_mean:.2f}/10")

**Analyse Graphique des Performances**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Configuration des graphiques
plt.style.use('default')
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Données pour les graphiques
categories = ['JPEG', 'Bruit', 'Filtrage', 'Géométrique', 'Contraste']
e_ad = [20, 100, 100, 40, 100]  # Valeurs moyennes approximatives
cihlhf = [1, 25, 3, 20, 45]     # Valeurs moyennes approximatives
cih_qr = [5, 15, 8, 50, 30]     # Valeurs moyennes approximatives

x = np.arange(len(categories))
width = 0.25

# Graphique 1: Robustesse comparative (BER moyen)
ax1.bar(x - width, e_ad, width, label='E-AD', color='red', alpha=0.7)
ax1.bar(x, cihlhf, width, label='CIHLHF', color='blue', alpha=0.7)
ax1.bar(x + width, cih_qr, width, label='CIH-QR', color='green', alpha=0.7)
ax1.set_ylabel('Taux d\'erreur (BER %)')
ax1.set_title('Robustesse Comparative (BER moyen)')
ax1.set_xticks(x)
ax1.set_xticklabels(categories)
ax1.legend()
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# Graphique 2: Performances temporelles
methods = ['E-AD', 'CIHLHF', 'CIH-QR']
dissimulation = [3689, 35.5, 696.7]
extraction = [0, 39, 646.6]  # E-AD n'a pas de mesure d'extraction

ax2.bar(methods, dissimulation, color=['red', 'blue', 'green'], alpha=0.7, label='Dissimulation')
ax2.bar(methods, extraction, bottom=dissimulation, color=['darkred', 'darkblue', 'darkgreen'],
        alpha=0.7, label='Extraction')
ax2.set_ylabel('Temps (ms) - échelle logarithmique')
ax2.set_title('Performances Temporelles (échelle log)')
ax2.set_yscale('log')
ax2.legend()
ax2.grid(axis='y', linestyle='--', alpha=0.7)

# Graphique 3: Capacité de stockage
capacity = [30752, 8192, 16384]
colors = ['red', 'blue', 'green']
ax3.bar(methods, capacity, color=colors, alpha=0.7)
ax3.set_ylabel('Bits théoriques')
ax3.set_title('Capacité de Stockage Théorique')
ax3.grid(axis='y', linestyle='--', alpha=0.7)

# Graphique 4: Score global comparatif
scores = {
    'Fonctionnalité': [2, 10, 10],
    'Robustesse': [2, 8, 7],
    'Capacité': [3, 6, 8],
    'Performance': [2, 10, 6],
    'Sécurité': [1, 8, 8]
}

categories = list(scores.keys())
e_ad_scores = [scores[cat][0] for cat in categories]
cihlhf_scores = [scores[cat][1] for cat in categories]
cih_qr_scores = [scores[cat][2] for cat in categories]

x = np.arange(len(categories))
ax4.bar(x - width, e_ad_scores, width, label='E-AD', color='red', alpha=0.7)
ax4.bar(x, cihlhf_scores, width, label='CIHLHF', color='blue', alpha=0.7)
ax4.bar(x + width, cih_qr_scores, width, label='CIH-QR', color='green', alpha=0.7)
ax4.set_ylabel('Score (0-10)')
ax4.set_title('Score Global par Catégorie')
ax4.set_xticks(x)
ax4.set_xticklabels(categories, rotation=45)
ax4.legend()
ax4.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('comparaison_steganographie.png', dpi=300, bbox_inches='tight')
plt.show()

**Diagramme Radar des Performances**

In [ ]:
# Diagramme radar pour comparaison visuelle
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, polar=True)

# Catégories pour le radar
categories = ['Fonctionnalité', 'Robustesse', 'Capacité', 'Performance', 'Sécurité']
N = len(categories)

# Scores normalisés (0-10)
e_ad_values = [2, 2, 3, 2, 1]
cihlhf_values = [10, 8, 6, 10, 8]
cih_qr_values = [10, 7, 8, 6, 8]

# Fermer le diagramme en reliant le dernier point au premier
e_ad_values += e_ad_values[:1]
cihlhf_values += cihlhf_values[:1]
cih_qr_values += cih_qr_values[:1]

# Calcul des angles
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Dessin du radar
ax.plot(angles, e_ad_values, 'o-', linewidth=2, label='E-AD', color='red')
ax.fill(angles, e_ad_values, alpha=0.1, color='red')
ax.plot(angles, cihlhf_values, 'o-', linewidth=2, label='CIHLHF', color='blue')
ax.fill(angles, cihlhf_values, alpha=0.1, color='blue')
ax.plot(angles, cih_qr_values, 'o-', linewidth=2, label='CIH-QR', color='green')
ax.fill(angles, cih_qr_values, alpha=0.1, color='green')

# Ajout des labels
ax.set_thetagrids(np.degrees(angles[:-1]), categories)
ax.set_ylim(0, 10)
ax.set_title('Diagramme Radar des Performances', size=14, y=1.1)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
plt.savefig('radar_steganographie.png', dpi=300, bbox_inches='tight')
plt.show()